In [273]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn import preprocessing
from settings import RAW_PATH,DATA_PATH
from src.utils import dump, loads,identity,unflatten
from src.funs import str_to_list, words_amount
import numpy as np
import time
from  datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer

from catboost import CatBoostRegressor, Pool

In [274]:
df_train = pd.read_csv(RAW_PATH /"train.csv", index_col= 0)
df_test = pd.read_csv(RAW_PATH /"test_v.csv", index_col= 0)

In [275]:
df_train_text = pd.read_csv(DATA_PATH /"df_text.csv", index_col= 0)
df_train_text = df_train_text[['text_length', 'avg_sentence_len', 'max_sentence_len', 'min_sentence_len', 'flesch', 'ari', 'cli', 'popularity', 'popularity_words']]
df_test_text = pd.read_csv(DATA_PATH /"df_text_test.csv", index_col= 0)
df_test_text = df_test_text[['text_length', 'avg_sentence_len', 'max_sentence_len', 'min_sentence_len', 'flesch','ari', 'cli']]

df_train_text_f = pd.read_csv(DATA_PATH /"df_text_co.csv", index_col= 0)
df_train_text_f = df_train_text_f[["img", "video", "a", "p"]]

df_test_text_f = pd.read_csv(DATA_PATH /"df_test_text_co.csv", index_col= 0)
df_test_text_f = df_test_text_f[["img", "video", "a", "p"]]

In [276]:
df_train = df_train.merge(df_train_text, left_index=True, right_index=True)
df_train = df_train.merge(df_train_text_f, left_index=True, right_index=True)
df_test = df_test.merge(df_test_text, left_index=True, right_index=True)
df_test = df_test.merge(df_test_text_f, left_index=True, right_index=True)

In [277]:
df_train.loc[df_train['full_reads_percent'] > 100, 'full_reads_percent'] = np.nan
df_train['full_reads_percent'].fillna((df_train['full_reads_percent'].mean()), inplace=True)

In [278]:
cat_le = preprocessing.LabelEncoder()
df_train["category"]=cat_le.fit_transform(df_train["category"])
dump(DATA_PATH/"category.pickle", cat_le)

In [279]:
auth_le = preprocessing.LabelEncoder()
df_train["authors"]=auth_le.fit_transform(df_train["authors"])
dump(DATA_PATH/"authors.pickle", auth_le)

In [280]:
df_train = df_train.apply(lambda row: str_to_list(row, 'tags'), axis=1)
tags_le = preprocessing.MultiLabelBinarizer()
tags=tags_le.fit_transform(df_train["tags"])
dump(DATA_PATH/"tags.pickle", tags_le)
tags_feat_names = ['tags_' + str(cls) for cls in list(tags_le.classes_)]
tags_df = pd.DataFrame(data=tags,
          index=df_train.index,
          columns=tags_feat_names)
df_train = df_train.merge(tags_df, left_index=True, right_index=True)
df_train = df_train.drop('tags', axis=1)

In [281]:
df_train['day'] = pd.to_datetime(df_train['publish_date']).dt.strftime("%d").astype(int)
df_train['month'] = pd.to_datetime(df_train['publish_date']).dt.strftime("%m").astype(int)
df_train['hour'] = pd.to_datetime(df_train['publish_date']).dt.strftime("%H").astype(int)
df_train['hour_binary'] = df_train['hour'].apply(lambda row: row >= 7 & row <= 17).astype(int)

df_train['timestamp'] = df_train['publish_date'].apply(lambda d:  time.mktime(datetime.strptime(d, "%Y-%m-%d %H:%M:%S").timetuple()))

In [282]:
def encode_l(row):
    text_length = row['text_length']
    if text_length<3_000:
        row['text_length']=0
    elif text_length>10_000:
        row['text_length']=1
    elif text_length>=3_000 and text_length<=7_000:
        row['text_length']=2
    else:
         row['text_length']=3
    return row
df_traiт = df_train.apply(lambda row: encode_l(row), axis=1)

In [283]:
df_train_ents = pd.read_csv(DATA_PATH /"df_text_ents.csv", index_col= 0)
df_train_ents = df_train_ents[['ents']]
df_train = df_train.merge(df_train_ents, left_index=True, right_index=True)
df_train['ents_len'] = df_train['ents'].str.len()
df_train = df_train.apply(lambda row: str_to_list(row, 'ents'), axis=1)

ents_le = preprocessing.MultiLabelBinarizer()
ents=ents_le.fit_transform(df_train["ents"])
dump(DATA_PATH/"ents.pickle", ents_le)
ents_feat_names = ['ents_' + str(cls) for cls in list(ents_le.classes_)]
ents_df = pd.DataFrame(data=ents,
          index=df_train.index,
          columns=ents_feat_names)
df_train = df_train.merge(ents_df, left_index=True, right_index=True)
df_train = df_train.drop('ents', axis=1)

In [284]:
ents_feat_names

['ents_LOC', 'ents_MISC', 'ents_ORG', 'ents_PER']

In [285]:
tags=['tags_5409f196e063daa0f408ae65', 'tags_546e40e7cbb20f1d52dbaa55', 'tags_5631bb949a794737108a5623', 'tags_5409f140e063daa0f408ad1d', 'tags_5e2af6549a794790a44be038', 'tags_5409f15ee063daa0f408adb0', 'tags_542d2d97cbb20fc67faf1128', 'tags_5409f15de063daa0f408ada7', 'tags_5409f168e063daa0f408add3', 'tags_56f117169a794719c3462090', 'tags_5409f1e4e063daa0f408af3b', 'tags_5409f1a2e063daa0f408ae94', 'tags_58abcf539a7947f18bc079eb', 'tags_5409f15de063daa0f408ada9', 'tags_546b54d1cbb20f84b19814b7', 'tags_543517b8cbb20fd884900e61', 'tags_5409f15de063daa0f408ada8', 'tags_59e769ab9a794784016dbf12', 'tags_5409f3c0e063daa0f408b40b', 'tags_5409f37be063daa0f408b2ae', 'tags_55797e339a794789759c64a1', 'tags_5409f324e063daa0f408b211', 'tags_5409f191e063daa0f408ae5c', 'tags_5409f145e063daa0f408ad35', 'tags_542589accbb20f99410da38a', 'tags_560588be9a7947241165f112', 'tags_621a3d0c9a794728d449ae5e', 'tags_5596ac4c9a7947422fd4b44c', 'tags_5576aa3a9a79471549e8b309', 'tags_5718b1a59a7947616a2028e8', 'tags_5444fea0cbb20f552bda691a', 'tags_5409f309e063daa0f408b1dd', 'tags_5409f173e063daa0f408aded', 'tags_5409f200e063daa0f408af68', 'tags_5409f16ae063daa0f408adda', 'tags_5433bdaacbb20f5239132963', 'tags_5462352bcbb20f235c8efea8', 'tags_54d6e7f69a7947648351f03b', 'tags_54f5a3f89a79472795e6331f', 'tags_5409f294e063daa0f408b111', 'tags_542ebd48cbb20f49621bd9a0', 'tags_58f35fe09a794726db128f47', 'tags_54c119f79a7947c2af17fab6', 'tags_5409f144e063daa0f408ad26', 'tags_545caa9ecbb20f366651641e', 'tags_543283d8cbb20f35cbd0f765', 'tags_5409f3b9e063daa0f408b3ce', 'tags_54da85b29a79472a52b4f2bc', 'tags_551062649a7947114b0b5179', 'tags_56fd0fda9a7947031dc422c4', 'tags_5426c60acbb20f76518c3e4c', 'tags_614cb40f9a79477066594352', 'tags_5444b07fcbb20f53feabf6ae', 'tags_5592a7c79a794719538cf7be', 'tags_54c78f629a7947b5345a6dfd', 'tags_546c644dcbb20feb6e4c2cb9', 'tags_5436346acbb20f86b01e0401', 'tags_5497170e9a7947f390f48648', 'tags_5448a1cecbb20fb6d148e7cb', 'tags_5409f198e063daa0f408ae6b', 'tags_5409f36be063daa0f408b286', 'tags_5c124ef99a79476e9d0352d3', 'tags_54371c54cbb20fd141c8bebd', 'tags_54cc317b9a79472a3fb95938', 'tags_58fda2219a7947718b2933e1', 'tags_55dd92239a794703a3a3d2ef', 'tags_55b0ce969a79475f572685bf', 'tags_546370b3cbb20fd4a15af908', 'tags_609e80e99a7947f9797f5844', 'tags_545c81a6cbb20fca592f6ed2', 'tags_5409f146e063daa0f408ad43', 'tags_54579d2dcbb20f99bc8c8cc1', 'tags_543d3460cbb20f079e577c75', 'tags_5409f15be063daa0f408ad97', 'tags_54e963e49a7947064ed50a58', 'tags_543ec15ecbb20f6e1fde87aa', 'tags_5adef5929a7947bbe6aab770', 'tags_5409f23ee063daa0f408b023', 'tags_542d1e28cbb20f862ac2c1f6', 'tags_5409f203e063daa0f408af72', 'tags_5409f1afe063daa0f408aeb7', 'tags_56fbdd1f9a7947725f756f96', 'tags_55b8acec9a7947303fa95eb2', 'tags_5409f414e063daa0f408b55e', 'tags_542ea011cbb20f42945c76b1', 'tags_54753030cbb20f5e2149daef', 'tags_54367e61cbb20ff87f5c52f1', 'tags_6204c5539a79474bfff916c3', 'tags_5409f3cae063daa0f408b456', 'tags_54c620339a794784d3c50327', 'tags_5409f15ae063daa0f408ad8f', 'tags_5e8ec7aa9a7947840f550498', 'tags_5450ef17cbb20fd39e3b9a63', 'tags_54366a01cbb20f810ba70f41', 'tags_547442e4cbb20f84324d17ff', 'tags_5409f3f2e063daa0f408b4c5', 'tags_5630b4289a7947e0e5785033', 'tags_5ea97d2f9a79474394271fd9', 'tags_5409f1f9e063daa0f408af59', 'tags_54662147cbb20fd1d7ffc15a', 'tags_544e3613cbb20f0c51fba35d', 'tags_5464a374cbb20f1979867cf7', 'tags_5463608ecbb20f33d60816f2', 'tags_547c0345cbb20f53267f531d', 'tags_591d9d909a79476a3d50e4a9', 'tags_569dcb249a7947f1b74ff5c3', 'tags_568b98d99a794790aa259f37', 'tags_57d7f9dd9a79472561094131', 'tags_5461d832cbb20f63a6d9b72c', 'tags_5980ff0f9a794701dc9bc80c', 'tags_5577f72f9a7947f6177effbc', 'tags_5444dcc5cbb20f4dc673525d', 'tags_54a12d0a9a794700b045f545', 'tags_60a684b39a794736366198d0', 'tags_54dca6bf9a794714bd831be7', 'tags_5551595b9a79472c8309e7a0', 'tags_54ec1fc89a7947086724adbe', 'tags_571f21e99a79472ea6ceef2d', 'tags_543e41aecbb20f294ca9dda5', 'tags_54dca6b79a794714bd831b8f', 'tags_545b75b3cbb20f967f3a1403', 'tags_615569a19a79475f841d75ac', 'tags_54378fb8cbb20f0dedb33594', 'tags_5409f1a5e063daa0f408ae9e', 'tags_5409f3a4e063daa0f408b34f', 'tags_543cfb50cbb20f4768f54642', 'tags_54483d84cbb20f42e1296da6', 'tags_5628d6629a79472116230197', 'tags_55ba51be9a794739e1610c32', 'tags_543cf054cbb20f05b7adb32f', 'tags_5453af87cbb20f4a65f19d13', 'tags_577d35be9a79479c09ce4ddb', 'tags_54cfc7e49a79479719452aa9', 'tags_5475f736cbb20f7c69461e88', 'tags_54685c70cbb20f596e3adada', 'tags_5409f1ffe063daa0f408af65', 'tags_5409f27ce063daa0f408b0c0', 'tags_56280f439a79472ea360ff61', 'tags_5409f257e063daa0f408b072', 'tags_542c628bcbb20f5b970dcd9f', 'tags_5409f235e063daa0f408affc', 'tags_5409f226e063daa0f408afd3', 'tags_542894c3cbb20f42017deb28', 'tags_5430fa87cbb20f41c21eb699', 'tags_55a75fca9a79476ef75918a9', 'tags_5409f1a3e063daa0f408ae98', 'tags_5409f289e063daa0f408b0da', 'tags_54ac0da19a7947ea8276b9cf', 'tags_5592b3239a7947d55bf524d5', 'tags_56a88f629a7947f5e7e9ce5f', 'tags_54bc460a9a7947264833234c', 'tags_54ba38649a79475e2f28abd9', 'tags_5409f1bee063daa0f408aed6', 'tags_5409f14fe063daa0f408ad68', 'tags_542e04b5cbb20f1f21570cba', 'tags_620612589a7947419847082e', 'tags_557751dc9a7947a9a529b191', 'tags_54257a93cbb20f012edf7750', 'tags_5426a035cbb20f7903d4f89f', 'tags_549a99a59a794711749dc8f2', 'tags_60051bdd9a79470a4b46173f', 'tags_5459ea7fcbb20f50286ad11c', 'tags_54289bd7cbb20f853ab53256', 'tags_549458ea9a794798759a326b', 'tags_5437f9eccbb20f615700f3b2', 'tags_599ecfca9a79472da7fab0dd', 'tags_5444f159cbb20f1b736b0303', 'tags_59e769a59a794784016db546', 'tags_560624289a794734e21d20a7', 'tags_54787519cbb20f1fd66203fa', 'tags_5409f401e063daa0f408b500', 'tags_5494134a9a79478b5126a2c5', 'tags_5409f15be063daa0f408ad93', 'tags_5592a9cd9a794719538d0ece', 'tags_550abd909a79476d38aa7672', 'tags_54601c22cbb20f10b7ca02e7', 'tags_5465d532cbb20f705cce70aa', 'tags_5409f29be063daa0f408b157', 'tags_5a3109c99a7947e23072968c', 'tags_54e59c4e9a79479a1300064e', 'tags_54b76d6e9a7947f9a10f1cac', 'tags_5509636d9a79471f686bc1aa', 'tags_5409f183e063daa0f408ae32', 'tags_5592a85f9a794719538cffcc', 'tags_5409f241e063daa0f408b029', 'tags_5571786e9a79477ee858d6d9', 'tags_5409f1b7e063daa0f408aec4', 'tags_5409f29ae063daa0f408b154', 'tags_562aabfd9a794741c377c954', 'tags_5409f195e063daa0f408ae63', 'tags_61c4adf49a794711387ad6ed', 'tags_55110d0e9a79475861c1cbdb', 'tags_54585dcdcbb20fb78d3e4db5', 'tags_556e926a9a794721a7851f18', 'tags_5409f3c9e063daa0f408b450', 'tags_559279af9a794733de001f76', 'tags_5409f31be063daa0f408b205', 'tags_54d3698e9a79476a580c8764', 'tags_596cb0199a7947386664c018', 'tags_5409f398e063daa0f408b315', 'tags_61fa82929a7947fca00326d4', 'tags_5409f336e063daa0f408b236', 'tags_545d4dcdcbb20fbc56178bed', 'tags_5493fec69a794789bf4cc40d', 'tags_59e769a69a794784016db80d', 'tags_567798469a794783dd5bf1c3', 'tags_54803f3bcbb20ff10c1134bf', 'tags_5658caca9a7947b13a301d58', 'tags_5409f163e063daa0f408adc7', 'tags_5773f5679a7947626a633a7c', 'tags_54262362cbb20f6397b0bd18', 'tags_5551b9de9a79475cf55f6156', 'tags_5409f174e063daa0f408adf9', 'tags_5409f23ce063daa0f408b01b', 'tags_5409f417e063daa0f408b578', 'tags_5ef9f4d99a79472ee0bd9751', 'tags_5409f1e0e063daa0f408af2e', 'tags_559294209a794751dc83a179', 'tags_5592b0809a794719538d4213', 'tags_550ebba29a7947ac919d5d3b', 'tags_5668323d9a7947ddc88702ff', 'tags_5409f1cae063daa0f408aeef', 'tags_58875dba9a7947daf6fbda12', 'tags_5409f17fe063daa0f408ae28', 'tags_559bc4279a79472c67243d18', 'tags_567813c99a79472690e75ea6', 'tags_545a3205cbb20f9a2d044db9', 'tags_5409f15ae063daa0f408ad8a', 'tags_5409f398e063daa0f408b312', 'tags_5409f1ede063daa0f408af4a', 'tags_5409f21ee063daa0f408afc1', 'tags_5409f2fae063daa0f408b1c4', 'tags_58a597382ae59633a5a257cf', 'tags_54986c919a79474919ad01f4', 'tags_54f6e36d9a7947e987d6c796', 'tags_60cc2b489a7947b296493579', 'tags_55928cb29a794751dc82f7c7', 'tags_55928d319a794751dc8303a1', 'tags_573809129a7947949c3016fe', 'tags_5409f297e063daa0f408b11c', 'tags_5555abae9a794766a68c03dc', 'tags_5430f451cbb20f73931ecd05', 'tags_58b6f4f29a79474561a7bc66', 'tags_59e769989a794784016d9ec9', 'tags_54dc856c9a7947798a4121d6', 'tags_54d1c0889a794742bd31d6e1', 'tags_546031facbb20f5e476e7725', 'tags_542d0fe4cbb20fdf04cd5adc', 'tags_59e769ab9a794784016dbf52', 'tags_5955eb469a7947689e6de3d6', 'tags_5409f3c9e063daa0f408b44a', 'tags_5409f429e063daa0f408b5d4', 'tags_54c0fdbc9a7947b019a42c0d', 'tags_543e39c6cbb20f360dd05574', 'tags_5436c247cbb20f090653f53e', 'tags_542a63e3cbb20f55b72dd71d', 'tags_5719f0409a79471199d3b420', 'tags_5409f23ce063daa0f408b01a', 'tags_601d0d649a79476f6587198c', 'tags_5433603acbb20f6e5def0cc5', 'tags_548ac7e92ae596d18ef45203', 'tags_563373789a794734713e0abd', 'tags_55b78e719a794762cb82f3fa', 'tags_5409f3cee063daa0f408b45d', 'tags_548fdc5c2ae59672aa925506', 'tags_55b5ce6f9a794771e876083e', 'tags_5409f3a0e063daa0f408b341', 'tags_542baab0cbb20fcd8bc5edba', 'tags_542a4aafcbb20feeac16f6d1', 'tags_5409f2e9e063daa0f408b1ab', 'tags_576012179a7947010084f0d5', 'tags_542bb984cbb20f7c4542e357', 'tags_5835d1169a79477e6bd1097b', 'tags_5409f37fe063daa0f408b2ba', 'tags_5435597acbb20fe010d5063f', 'tags_5409f41de063daa0f408b589', 'tags_55928f739a794751dc833c3a', 'tags_5409f422e063daa0f408b5ae', 'tags_55dc8a529a7947cd370c3541', 'tags_5e8ec7799a7947848604a99e', 'tags_61d87adf9a7947df81912fa7', 'tags_54554036cbb20fb530f83218', 'tags_5470d0a0cbb20fec40b24594', 'tags_5409f2fce063daa0f408b1cd', 'tags_5409f17be063daa0f408ae13', 'tags_54365ce9cbb20fecb0becca6', 'tags_548b1d422ae596424c8a515d', 'tags_5449f78dcbb20f7164fea005', 'tags_56a0a6c29a7947b20a9d37ef', 'tags_5409f28de063daa0f408b0e2', 'tags_5733af7c9a79475caed7d43f', 'tags_58988a599a794775cda0cf88', 'tags_544502dacbb20fa7872b408a', 'tags_54661726cbb20f44126635aa', 'tags_572b80f59a7947bfecbb044d', 'tags_544f98f8cbb20f22f0212d2e', 'tags_5409f413e063daa0f408b556', 'tags_54272933cbb20f0b0236068b', 'tags_5434150fcbb20fe5dafc801b', 'tags_60fe45019a794710abb5c256', 'tags_5592a8a39a794719538d033c', 'tags_560e57129a7947d153e4cfd4', 'tags_5409f37be063daa0f408b2aa', 'tags_561bb26a9a7947a27717c0a1', 'tags_542c454ecbb20f6099323d4a', 'tags_54826f44cbb20f7621273a2f', 'tags_54cd09869a79475c927824e0', 'tags_5409f399e063daa0f408b31c', 'tags_54315649cbb20f67c5046e28', 'tags_544e640bcbb20fa086430155', 'tags_54a1d3969a794719665cdda6', 'tags_56c486869a7947bec85e6ae3', 'tags_5409f205e063daa0f408af77', 'tags_54c6056a9a79476831284ec2', 'tags_560412f49a794793ea3dd444', 'tags_543452a9cbb20f9c392925e3', 'tags_5964bf769a7947b930a599fa', 'tags_5409f2c9e063daa0f408b17d', 'tags_54257a76cbb20f62c705edf2', 'tags_54e0c1019a7947ea17b6bed9', 'tags_55069f189a7947f24c29d989', 'tags_5409f2c9e063daa0f408b17e', 'tags_5409f19be063daa0f408ae7c', 'tags_54631a48cbb20f627a790eab', 'tags_55f311039a79477ea68dca56', 'tags_54cf0f409a79472e4c1dba3d', 'tags_59e7699d9a794784016da80e', 'tags_5762a9f29a7947122b458519', 'tags_54288ee0cbb20f681cbb3e9d', 'tags_54367e1ecbb20ffdb159de4e', 'tags_55928d199a794751dc8300f3', 'tags_551656679a794722a8dc063c', 'tags_561bf0359a7947ec2eaaf789', 'tags_5409f423e063daa0f408b5b7', 'tags_543544f6cbb20f6efbd0ee3a', 'tags_5409f3a5e063daa0f408b354', 'tags_563388ca9a79475cce38995a', 'tags_61a1fd729a794761d34b30b7', 'tags_578490e49a79474c51627097', 'tags_547d9fc5cbb20f7d39cad31c', 'tags_569ef8979a7947a12cc70147', 'tags_5640c7149a79477ff8d2f5e0', 'tags_56a9a2679a7947665340a9a8', 'tags_544df6d3cbb20f858920ef13', 'tags_544e300dcbb20f215a0d3293', 'tags_54266f90cbb20f90d7881129', 'tags_54432619cbb20f87865683d0', 'tags_58be9b0f9a7947a8aebeb440', 'tags_5429372bcbb20f106a5f76a9', 'tags_55ba10489a7947cf559e3806', 'tags_5409f160e063daa0f408adb8', 'tags_5a2ec68a9a7947b090ace3c1', 'tags_5409f3f1e063daa0f408b4bd', 'tags_599a399d9a794742b80d6c15', 'tags_5409f18be063daa0f408ae49', 'tags_5409f306e063daa0f408b1db', 'tags_549a9c689a79471202397f9b', 'tags_542562d5cbb20f6fff7ee57d', 'tags_545e78b7cbb20f9974e104ef', 'tags_559294929a794751dc83a990', 'tags_542f57c3cbb20fb76f1b72f4', 'tags_5592ab559a794719538d2137', 'tags_5463e5b9cbb20f877fdedd71', 'tags_5409f19be063daa0f408ae7a', 'tags_59ef45809a7947efbccd5474', 'tags_544f0d6acbb20f7c272187f4', 'tags_54cdb95a9a7947746817541f', 'tags_5409f175e063daa0f408ae02', 'tags_542ebbc0cbb20f35dc1f5305', 'tags_55abdafb9a79473677d818d4', 'tags_56331b2c9a794751a4177b2b', 'tags_5479a24fcbb20f3bc050403d', 'tags_5440feebcbb20f9d2a870667', 'tags_56158b559a79475e02a1ce2f', 'tags_544e4ba3cbb20f0e847b1990', 'tags_5409f15ae063daa0f408ad8c', 'tags_5451eb6bcbb20f340d3062c4', 'tags_543f859fcbb20f5878ef7436', 'tags_54cb7b789a7947e87ac45e10', 'tags_54c8a9bd9a79472b709bd721', 'tags_5409f151e063daa0f408ad71', 'tags_58ac18359a79471245f601ea', 'tags_5a2162169a7947720f9323c6', 'tags_553df6e29a79477ed605074f', 'tags_5409f145e063daa0f408ad30', 'tags_5592aa469a794719538d149d', 'tags_543cc09acbb20f9cdd962ef7', 'tags_5409f19ce063daa0f408ae81', 'tags_542a1cbccbb20f385143e019', 'tags_54b974a59a794748a0224e69', 'tags_54481740cbb20f6a62117099', 'tags_54333876cbb20f2b9632eaec', 'tags_5430fa87cbb20f41c21eb698', 'tags_55928da09a794751dc830eb7', 'tags_5439d1fccbb20f0850116006', 'tags_5409f190e063daa0f408ae56', 'tags_5425598acbb20f7a3ba6ae56', 'tags_5409f384e063daa0f408b2c2', 'tags_580745619a7947afcc592130', 'tags_546c8c88cbb20f379f985645', 'tags_5450278ecbb20f75bae8d827', 'tags_555a2aca9a7947b513c4714f', 'tags_592e6ce49a7947a7852bb5c8', 'tags_54c127b69a7947c9188d5c00', 'tags_545ce7a3cbb20f02192fb141', 'tags_5409f425e063daa0f408b5bf', 'tags_5523ca509a7947e3dc52818c', 'tags_5431e747cbb20f65bf8e5cb3', 'tags_59edc6959a794707031e513c', 'tags_543e360fcbb20ffa39153d5b', 'tags_545f4070cbb20fd610351148', 'tags_54dc84989a79477970b4fbac', 'tags_54a2689b9a7947335b710337', 'tags_5409f19be063daa0f408ae78', 'tags_59e769a49a794784016db45c', 'tags_54cf8b859a794771252ddf0b', 'tags_566951239a79479eb17ab51f', 'tags_620690a49a79478565a0da0f', 'tags_54b9190e9a79473dd12dec86', 'tags_5a057b4d9a794716f11abf45', 'tags_554bee069a794702d9f66096', 'tags_5424b76acbb20feba0485f4d', 'tags_5447d958cbb20f05db166907', 'tags_54341503cbb20fe258a7603f', 'tags_56fd09719a7947fb6842881e', 'tags_5c66a9ff9a7947a23491199d', 'tags_54333890cbb20f2b9632eaee', 'tags_5458821ecbb20feed14fd08a', 'tags_5409f16be063daa0f408addc', 'tags_5433f8cacbb20f1662a20a83', 'tags_543e442acbb20f4211ae4784', 'tags_547c27cfcbb20fd805beae41', 'tags_54a7f1ea9a794759cca91e8c', 'tags_54272eedcbb20f67193b15bf', 'tags_542572c1cbb20f2a44a6acca', 'tags_54249b0fcbb20feba0485f3e', 'tags_550070469a7947fc3e92781e', 'tags_54325c99cbb20f7eb79a0415', 'tags_546e717acbb20fe8e57361ee', 'tags_54dc856d9a7947798a4121eb', 'tags_54325710cbb20f2aee7c7a0d', 'tags_59e769a29a794784016dafcb', 'tags_59e769a59a794784016db5a7', 'tags_55928d959a794751dc830d97', 'tags_5409f3c2e063daa0f408b417', 'tags_54db29aa9a794754480ef239', 'tags_5409f250e063daa0f408b05d', 'tags_54d0997b9a7947c72f2b7b73', 'tags_5409f254e063daa0f408b068', 'tags_54c7a1959a7947bc19dc0bff', 'tags_556761ef9a79470e3ddb84cd', 'tags_5a19de899a7947ebd8144ec0', 'tags_54dc84879a79477970b4fa8e', 'tags_59e769a49a794784016db2f1', 'tags_55928ba99a794751dc82e1e9', 'tags_58ca77679a794700b1e37f0a', 'tags_559293799a794751dc83946c', 'tags_5445ffa3cbb20f31af6847d4', 'tags_549ab4fa9a79471822c49545', 'tags_5409f208e063daa0f408af83', 'tags_5f0579399a7947110466355e', 'tags_6245e83d9a79470666cc7661', 'tags_5409f2fae063daa0f408b1c5', 'tags_54523c29cbb20f39f4ab33e9', 'tags_542bc4a4cbb20f7dd040b517', 'tags_54297c15cbb20f3b894ccc0d', 'tags_5439b956cbb20f64d0cc5958', 'tags_564c7f289a79474e4fb94cb3', 'tags_543cf58bcbb20f7c134e1470', 'tags_5409f3bae063daa0f408b3dd', 'tags_61c5d5619a794775b0582c05', 'tags_58b9a72d9a79474b530f973b', 'tags_6022cc509a794709b1aaa7f1', 'tags_54eeecea9a79472768071c78', 'tags_5731e7589a7947e8310f6665', 'tags_5409f35de063daa0f408b273', 'tags_5558cdd59a7947514f9f1733', 'tags_54d78cf89a794707bfb61eb5', 'tags_5562eee49a7947eab911ea59', 'tags_5efb5a5b9a79473bf986d838', 'tags_559e8b519a79478243c22f59', 'tags_5409f377e063daa0f408b2a4', 'tags_5450c4f2cbb20fe5f23a195a', 'tags_5444107bcbb20f054a121a8c', 'tags_542bf061cbb20fb2e1d476f7', 'tags_549c41bb9a794753c5145052', 'tags_559296909a794751dc83c8af', 'tags_5592920d9a794751dc8377b8', 'tags_5409f3abe063daa0f408b386', 'tags_554e151c9a7947660301015b', 'tags_5409f168e063daa0f408add2', 'tags_5409f1a5e063daa0f408aea0', 'tags_559293bd9a794751dc8399d3', 'tags_5409f19ce063daa0f408ae80', 'tags_542a7b08cbb20f28f0bf09b1', 'tags_5465eb84cbb20fe37d7b488d', 'tags_589085ca9a794761341dce1e', 'tags_54856f72cbb20fc24f8ac63c', 'tags_5409f42be063daa0f408b5e0', 'tags_5409f2b0e063daa0f408b171', 'tags_5592b2b49a7947d291915cf4', 'tags_54e362789a794761ccd07e0b', 'tags_5fe0a0fc9a79479285e4483c', 'tags_544e640bcbb20fa086430153', 'tags_5454bd27cbb20f7c1daddc8a', 'tags_544e80cccbb20f616634720f', 'tags_5433e218cbb20f863a189888', 'tags_546a83cdcbb20f462386cfec', 'tags_5481d5dacbb20fd58557d64d', 'tags_56b0c1249a79477ad0177aad', 'tags_5836ab989a7947702752a22e', 'tags_553a29679a794751d370fa4e', 'tags_559291019a794751dc8360f9', 'tags_54c0bb529a79478bb9a34e6c', 'tags_54272eedcbb20f67193b15be', 'tags_5582b2039a7947e9600a8154', 'tags_5459fe3acbb20f4b28c2f5a9', 'tags_55e717b29a7947fb9791fdf4', 'tags_55232d559a79479cee5330e9', 'tags_58ebb2699a794757edf778fc', 'tags_551017079a79475ada8509ce', 'tags_554c983b9a794745e96a3320', 'tags_544f87c8cbb20f9821503f7f', 'tags_54b644da9a7947d1087e29d4', 'tags_54f80ba69a7947200969dd41', 'tags_54fef9239a7947206bee5732', 'tags_548725d7cbb20f27190c1bcf', 'tags_5409f429e063daa0f408b5d3', 'tags_572b7e739a7947bd9e5bf816', 'tags_5409f416e063daa0f408b573', 'tags_5437e16ccbb20fc624878b02', 'tags_5696d22f9a794755da1b9935', 'tags_61dfef379a794706f1c5c779', 'tags_54263165cbb20fc5cc82528e', 'tags_5428d8aecbb20f531f21a38c', 'tags_598d90719a7947b0cc6cc2cd', 'tags_55bbbbf59a7947ca179a15a3', 'tags_5409f3e6e063daa0f408b495', 'tags_542bf8bfcbb20f79d4c9cdd3', 'tags_5959ed0b9a79473fdad66859', 'tags_561535609a7947f0584227bd', 'tags_54254853cbb20f240250ba32', 'tags_5429563acbb20f0ce315fb1d', 'tags_59e7699a9a794784016da172', 'tags_5a1866fc9a79475a04804f66', 'tags_542cce5dcbb20fb5c1103c63', 'tags_54dcaa7d9a79471bacf43b6c', 'tags_5a0011ef9a79470fc2cb1ee0', 'tags_5457db2acbb20f17cc6361ed', 'tags_559518409a79477ab4479bec', 'tags_5432a369cbb20f3f1d83336c', 'tags_5409f30fe063daa0f408b1f1', 'tags_582ec4329a79474ad83c3a98', 'tags_542c146bcbb20f402910e2b6', 'tags_54a3bc719a79475cf558319d', 'tags_54cb8efa9a7947efd40997dc', 'tags_5409f270e063daa0f408b0a7', 'tags_5409f191e063daa0f408ae5f', 'tags_54dc84a99a79477970b4fc83', 'tags_54374f9ecbb20f0fab07f3a6', 'tags_59e7699a9a794784016da15b', 'tags_54749e7acbb20f27dfbf3d9d', 'tags_546b1f01cbb20f253aeae866', 'tags_5503e37b9a7947531f2e5b76', 'tags_559292709a794751dc837ff7', 'tags_59c1ecba9a7947b1307aa1c7', 'tags_5409f1bbe063daa0f408aece', 'tags_5409f196e063daa0f408ae67', 'tags_54eca5e79a79477111c29526', 'tags_5424c125cbb20f353dbe036a', 'tags_542c94fccbb20f3a7b29e9cf', 'tags_59e769a99a794784016dbbe2', 'tags_544722eacbb20f3473e6ffe3', 'tags_5fae38aa9a7947769a0aeac4', 'tags_542caa49cbb20fab01715ae7', 'tags_5592b2d79a7947d55bf51cb0', 'tags_5409f30ee063daa0f408b1e9', 'tags_543e39c3cbb20f44458c926e', 'tags_5447cbddcbb20f554b5fd65d', 'tags_5427a33acbb20f0fc60a86a4', 'tags_5592b32e9a7947d55bf525c0', 'tags_55928ef49a794751dc83303a', 'tags_54520571cbb20f1259827b19', 'tags_5409f3a6e063daa0f408b35f', 'tags_5409f30ae063daa0f408b1e0', 'tags_5409f176e063daa0f408ae05', 'tags_546393f9cbb20f4495d40451', 'tags_551709d89a79477785992f1a', 'tags_5409f1c0e063daa0f408aedd', 'tags_5409f3b2e063daa0f408b3b7', 'tags_624d6bbc9a7947413e92d2a6', 'tags_5409f3f7e063daa0f408b4dd', 'tags_565837a09a79471beeaa9cfc', 'tags_5409f1a9e063daa0f408aea5', 'tags_5883d50f9a7947eac61a7dc5', 'tags_54afcb049a7947629823e6ef', 'tags_549566c69a7947b20efaa8d3', 'tags_54b1abef9a79472b340d3e9e', 'tags_56531e419a794744512f46df', 'tags_58e624d29a7947408a6d10e8', 'tags_5470c204cbb20fa2f4f67a61', 'tags_54d510c19a7947184f116809', 'tags_59bd2c2c9a79471228485aeb', 'tags_542580eacbb20f48618cb28b', 'tags_551d65f19a7947f535b9c172', 'tags_5409f3bfe063daa0f408b405', 'tags_5592951b9a794751dc83b326', 'tags_5409f1b9e063daa0f408aeca', 'tags_54366a01cbb20f810ba70f42', 'tags_542d866ecbb20f60c91a63a7', 'tags_54f45b0c9a7947daf893194d', 'tags_5526c0c19a794731185f79e3', 'tags_542b5794cbb20f7c0ab652ca', 'tags_5409f25ae063daa0f408b07c', 'tags_559a16e59a7947473fdc8840', 'tags_548d87362ae596f016401a5b', 'tags_54d9ca719a7947cc127170f8', 'tags_5429300dcbb20f177ed686e5', 'tags_5409f298e063daa0f408b126', 'tags_56a8d4209a7947527b47072d', 'tags_5409f271e063daa0f408b0aa', 'tags_544c0988cbb20f7d8fb316be', 'tags_546796c9cbb20f0f7f86e0dc', 'tags_542bbdcacbb20f1c64d1705a', 'tags_586226239a7947f55f264c56', 'tags_54f9d33b9a79472f74f6969b', 'tags_5409f29ee063daa0f408b15b', 'tags_562a0a909a79471aa841cfb3', 'tags_544bc4e5cbb20f11c2d27812', 'tags_54858b45cbb20faea9a102e9', 'tags_5409f3a7e063daa0f408b368', 'tags_5409f254e063daa0f408b069', 'tags_5448f8aacbb20f9b5a49ada1', 'tags_624bd8289a794736f740318a', 'tags_5465eb85cbb20fe37d7b488e', 'tags_542891becbb20f16ec453e23', 'tags_56606a4c9a7947f8f61dfd50', 'tags_542b9627cbb20f572a4d3a9f', 'tags_57d292d09a7947a797412afd', 'tags_56089e479a79479e21d5e41b', 'tags_54268824cbb20f4122da4b1c', 'tags_542b3d10cbb20f47c07dcb55', 'tags_55b228929a7947605e3d4081', 'tags_59c1149c9a79472513272d36', 'tags_5497d4809a79470cb322614c', 'tags_581c55509a7947394b329d5c', 'tags_543e6c6acbb20fa8f2813054', 'tags_5409f1abe063daa0f408aeab', 'tags_61e04b909a79473aad43f553', 'tags_5409f22fe063daa0f408afe2', 'tags_564337909a794750393dc381', 'tags_54dcaa809a79471bacf43b98', 'tags_55505daa9a79473fd4c8fcc3', 'tags_5409f16be063daa0f408addd', 'tags_59930b349a79470d55a94847', 'tags_5409f3a0e063daa0f408b342', 'tags_5409f3bae063daa0f408b3d4', 'tags_54464bd7cbb20f9dcd2237f1', 'tags_542687bdcbb20f33d870523c', 'tags_549a79c49a79470c8d863e8b', 'tags_5444a9c5cbb20ffed561eecb', 'tags_54b916449a79473afd583a39', 'tags_5409f260e063daa0f408b08f', 'tags_54ba530c9a79476763430c0f', 'tags_54dca6b09a794714bd831b46', 'tags_54bcdb309a794736bb6412f1', 'tags_5409f19de063daa0f408ae86', 'tags_5915c4b69a7947500fe4e3c6', 'tags_5409f406e063daa0f408b510', 'tags_55928f409a794751dc833759', 'tags_544e08cccbb20fdfb429b81a', 'tags_5592aaf89a794719538d1ccd', 'tags_55a39edf9a794728b373c81e', 'tags_543446d0cbb20f158e77b4c6', 'tags_5409f15ce063daa0f408ad9f', 'tags_562e28f69a79475a3cbaf75d', 'tags_544fc6d6cbb20f68ebc8f49f', 'tags_54b541379a7947ab1d390560', 'tags_5459f694cbb20fe0c93555ae', 'tags_54504735cbb20f531dc20031', 'tags_545b9c17cbb20fa1a934a1d9', 'tags_542bf4a9cbb20f4428e68635', 'tags_549dc43e9a79478c303bc12e', 'tags_543ea51acbb20f3f46c10e07', 'tags_5763e5a79a7947155046f7bd', 'tags_5e8e5f0b9a79476118e3d7f8', 'tags_571653869a794746be16bd9a', 'tags_546b34efcbb20f595858e800', 'tags_544fa29bcbb20f56bf6b11df', 'tags_5450ccc6cbb20f8bc5804d86', 'tags_54783c7ccbb20f6db32e5009', 'tags_54b3ea0a9a7947768fab5f0d', 'tags_5452416ecbb20f7a50a90408', 'tags_55238d9c9a7947c219a091d8', 'tags_56a74f9f9a79473c0f4b2f84', 'tags_55928e5e9a794751dc8321b2', 'tags_544db4d7cbb20f04a29c33ed', 'tags_5424bb11cbb20f96c9a63b76', 'tags_5409f17ce063daa0f408ae1b', 'tags_5433d0b3cbb20fb2a5a2cae8', 'tags_544ab95bcbb20f5bc127601b', 'tags_5444c29ecbb20f79317a24ad', 'tags_543e87eccbb20f3eb3401ce8', 'tags_5592a9439a794719538d0802', 'tags_588b07f19a794731b1ca815f', 'tags_59e769a69a794784016db83d', 'tags_5409f162e063daa0f408adc2', 'tags_551019fd9a79475bf1be54b6', 'tags_54de432e9a7947c1ed179b13', 'tags_5409f234e063daa0f408aff6', 'tags_542d93e9cbb20ffd69e280d3', 'tags_56cedb8d9a79475f241fc364', 'tags_54838946cbb20ff4b81d85b0', 'tags_5409f414e063daa0f408b55f', 'tags_54a50b549a7947685a73f184', 'tags_55928c5f9a794751dc82f047', 'tags_548df1a72ae5966895b9df7b', 'tags_542e77dccbb20ffaecf4f512', 'tags_548061e5cbb20ff0fdb3c8fb', 'tags_5409f3a9e063daa0f408b36c', 'tags_5592aac69a794719538d1a63', 'tags_594b5c099a7947ae69fc224e', 'tags_549dc00a9a79478c22e4b94f', 'tags_594b3c139a79479f9ff0a473', 'tags_5425a76dcbb20fce469cf538', 'tags_5464dfdbcbb20f75378e547b', 'tags_59e769a49a794784016db45f', 'tags_560f30149a7947630db65a83', 'tags_54dc84999a79477970b4fbc0', 'tags_553d9fce9a794744db87ccd4', 'tags_543b9e31cbb20fd1a760b711', 'tags_5409f160e063daa0f408adb9', 'tags_5432c059cbb20f1b72fe4201', 'tags_598ad3f39a79477766335296', 'tags_549e9d9a9a7947ab4a7032e5', 'tags_55fabb6e9a7947e80c43ea31', 'tags_5409f1ebe063daa0f408af42', 'tags_5447f4e0cbb20f9dfda9f90f', 'tags_542aec3dcbb20fda8fe109ba', 'tags_544fe0accbb20f32b14924ef', 'tags_597756199a79474420caf05f', 'tags_564cb4d69a79473a5d094e9c', 'tags_543abdc2cbb20f1886eedda6', 'tags_5409f41de063daa0f408b58a', 'tags_5593d8509a7947915e15ff7a', 'tags_55d84e2b9a794706f056dadf', 'tags_5409f2a7e063daa0f408b167', 'tags_59e769a19a794784016dad27', 'tags_55c824c19a7947f71e40ed95', 'tags_5469e78fcbb20f759861bdab', 'tags_543a73c0cbb20fc674279373', 'tags_567a91c29a79479b130aaebb', 'tags_5409f2fce063daa0f408b1d2', 'tags_5592a7309a794719538cefef', 'tags_553f94da9a7947102ffc53e8', 'tags_5473c986cbb20f7b9cdca5a3', 'tags_546065d3cbb20ff82b1ced47', 'tags_564dbeb89a7947efa3152daf', 'tags_551ceb5f9a7947a7fdb311b5', 'tags_5602952b9a79474847d26eec', 'tags_5592a7659a794719538cf2e6', 'tags_5409f264e063daa0f408b09a', 'tags_5409f1c1e063daa0f408aee1', 'tags_59e769a89a794784016db9e4', 'tags_5409f198e063daa0f408ae6c', 'tags_542d097fcbb20f8b63ae1572', 'tags_559a59139a7947847836c77d', 'tags_54b761589a7947f9a10f1ca1', 'tags_545b78d4cbb20f9df71de815', 'tags_543544c5cbb20f735ae0ebe2', 'tags_547c1516cbb20f53267f5323', 'tags_54353a0ccbb20ff2f73e2714', 'tags_5459f3e9cbb20f688165354b', 'tags_543e9ac9cbb20f2f475c02e6', 'tags_6277deb09a79472bf2dff2a7', 'tags_56cdd2e69a79478498db7dcc', 'tags_5562eee49a7947eab911ea58', 'tags_58ef9ae99a7947246a100c9a', 'tags_5538b3d79a794776bd9db70e', 'tags_54dc85619a7947798a412153', 'tags_559292db9a794751dc8388b0', 'tags_542a765ccbb20f7c128289c9', 'tags_55608f469a79476c0c4e0864', 'tags_5409f1ffe063daa0f408af66', 'tags_546b4c85cbb20f30a8ff0b80', 'tags_5409f252e063daa0f408b062', 'tags_56581ffb9a794784e6986495', 'tags_5592a68f9a794719538ce56c', 'tags_542964b6cbb20fb91ee52079', 'tags_5409f293e063daa0f408b104', 'tags_5433fd8bcbb20ff8503c3769', 'tags_547de4f6cbb20f529451b005', 'tags_543f70c2cbb20f824417ebc1', 'tags_582546689a794750649b3a6a', 'tags_55a24be99a7947eaacd44447', 'tags_628774149a7947522fa2ef8d', 'tags_54ef0c319a7947453eeb641d', 'tags_547523cecbb20f3ca7a15211', 'tags_546240cdcbb20f8a5c90571b', 'tags_56fe8bbd9a79472e634d0d6f', 'tags_563ca1999a79478ccce45480', 'tags_553f5cc19a7947e02931e72d', 'tags_54d47f159a7947dc04c6bbe6', 'tags_55928da79a794751dc830f8d', 'tags_5efee4539a7947094e1e5bea', 'tags_5592b2ac9a7947d291915be6', 'tags_54a3d29e9a79475f4f14a78a', 'tags_5409f38be063daa0f408b2c6', 'tags_55e4ff879a7947ac483df5d5', 'tags_5489661a2ae5969de17286e8', 'tags_575695d89a794750ab15f5ac', 'tags_59c30fcb9a794722ce73cd29', 'tags_589c29789a79473c5a92f639', 'tags_587de9339a7947571502ba97', 'tags_59e769ab9a794784016dbf6c', 'tags_5486d4aecbb20fdbd81d469d', 'tags_5409f1e1e063daa0f408af36', 'tags_54b2bc079a79474aab0b34fc', 'tags_553a43339a79476595cb8bd6', 'tags_55686a349a7947102532e138', 'tags_593f894c9a7947a62b6c906c', 'tags_546afed5cbb20f09c41de891', 'tags_5592a7749a794719538cf3a7', 'tags_58eb79279a794790272ed0fe', 'tags_549bc3929a794737b25e4455', 'tags_547c549ecbb20f3a16d83819', 'tags_544e1f8fcbb20f7c92ac4e1b', 'tags_545b7e03cbb20fde98102a6b', 'tags_54732716cbb20f2a137204ec', 'tags_56086a089a79479898721339', 'tags_5527bd7d9a794709e3d73b28', 'tags_54639176cbb20f32da66e1d5', 'tags_59e7699d9a794784016da7c5', 'tags_55928ef59a794751dc83305a', 'tags_5592b47a9a7947de211fce6e', 'tags_5409f3a5e063daa0f408b35b', 'tags_5592ab9f9a794719538d2485', 'tags_54dc84909a79477970b4fb2d', 'tags_5409f364e063daa0f408b27d', 'tags_54c1106a9a7947b99bf756b7', 'tags_551d10659a7947bad0961977', 'tags_5409f29ae063daa0f408b151', 'tags_54621d74cbb20f65713b6ccf', 'tags_542ea0cfcbb20f6c96675c7c', 'tags_544a4279cbb20fadc4c42182', 'tags_545ccf1acbb20f45a1f61c78', 'tags_5525c3d59a7947719eae2ffe', 'tags_569ced879a7947382935cf12', 'tags_584ed9d89a79475363f9245d', 'tags_5447565dcbb20feebd16f565', 'tags_5409f174e063daa0f408adfd', 'tags_5433d189cbb20fba1b242f30', 'tags_5434a7d6cbb20f0883bcf33a', 'tags_54a1e4cd9a79472092abee07', 'tags_5434dceecbb20ff0fc9131e4', 'tags_5592a71a9a794719538cee5c', 'tags_55a63f0e9a794793b43687e7', 'tags_5409f40ce063daa0f408b529', 'tags_543f272ecbb20f1a4c0919f4', 'tags_543d0026cbb20f7c127eaef8', 'tags_5409f145e063daa0f408ad31', 'tags_542a57c8cbb20f06f6b2fcab', 'tags_54dc858a9a7947798a412372', 'tags_54748ba6cbb20f87ae9b7c13', 'tags_5409f38fe063daa0f408b2e5', 'tags_5444ec9bcbb20f20bf739a47', 'tags_5592b4329a7947de211fc771', 'tags_5592b06b9a794719538d417d', 'tags_5409f3e3e063daa0f408b48a', 'tags_55928d509a794751dc83069a', 'tags_54254296cbb20f71947b9e01', 'tags_574021359a7947e5302e0376', 'tags_56e9ec0e9a7947fc70c3e8a9', 'tags_5592b0809a794719538d4208', 'tags_5437b689cbb20f5a5f33a078', 'tags_59089c189a7947eed80bd18a', 'tags_542b5794cbb20f7c0ab652cb', 'tags_59e769a89a794784016db926', 'tags_54d824559a794734c8870a48', 'tags_5447c3f0cbb20f7d2ab7684f', 'tags_542673e4cbb20fee51ea5523', 'tags_59de617a9a794753628e7ca5', 'tags_54379affcbb20f9bb478ba14', 'tags_55ac97ed9a79470cf7ad3a24', 'tags_563af5ed9a79479f95caa249', 'tags_56bae5b69a79476c5c7b3273', 'tags_55f2defb9a79472cc57b4f42', 'tags_5782d28e9a79471ac8bf5fbd', 'tags_5409f175e063daa0f408ae00', 'tags_57ce9ef19a7947293704f13d', 'tags_54435596cbb20f7043847585', 'tags_55de25ce9a7947989f4cb166', 'tags_54291ac8cbb20f3bbde756e8', 'tags_5409f408e063daa0f408b51e', 'tags_542a7349cbb20f5893eb2adf', 'tags_5608e2d59a794711bcc0bcda', 'tags_5482395ecbb20f89c555a9a3', 'tags_54280e18cbb20f7c0baefe17', 'tags_5592a7bd9a794719538cf73b', 'tags_55a8ea1a9a794756c5c3cc66', 'tags_5571ea639a7947d496d79f5c', 'tags_54e6dd729a794765c2ce6713', 'tags_5409f21be063daa0f408afb5', 'tags_5555b8f89a79476cac8203e6', 'tags_62110a649a7947f74b4c95b8', 'tags_54bf49199a79474ef77de105', 'tags_5f92dade9a794762939576c2', 'tags_584550a29a79477e9f9ff8f9', 'tags_5426f065cbb20f95cd885fe2', 'tags_5447dee9cbb20f25a3ceadb5', 'tags_559294979a794751dc83a9ef', 'tags_5409f1ebe063daa0f408af41', 'tags_553dcf9c9a794764dd8fae07', 'tags_5409f3e2e063daa0f408b483', 'tags_560a7d5e9a7947f252239bcc', 'tags_55928fdb9a794751dc83463c', 'tags_5409f20ee063daa0f408af99', 'tags_5772193f9a79479b2abecdbc', 'tags_542e4a24cbb20f4eb1d231a9', 'tags_54bf7d449a79475c6562b59e', 'tags_5592a72e9a794719538cefc7', 'tags_5749aadd9a79479db333799e', 'tags_5592a8059a794719538cfb1c', 'tags_5429fd05cbb20f4f66c3d574', 'tags_5409f1c3e063daa0f408aee7', 'tags_5409f3eee063daa0f408b4b3', 'tags_5592a7dd9a794719538cf8e5', 'tags_54bbecc09a794715a90bec84', 'tags_5592a7959a794719538cf533', 'tags_55307d9c9a79475ad63559c5', 'tags_565efae69a79479cefeb69af', 'tags_54c1de319a794779f817b208', 'tags_542d3116cbb20ff2ab2ceaae', 'tags_553f994c9a79471b920042bb', 'tags_54c9f9269a79474490d5f56c', 'tags_548f9d8d2ae59686876519a9', 'tags_54fdd4909a79471bb3fcdfd1', 'tags_559290ad9a794751dc835977', 'tags_5592a92d9a794719538d06d9', 'tags_56e705999a79472738570566', 'tags_5409f191e063daa0f408ae5e', 'tags_548f08992ae596485d06cb0a', 'tags_542d0c99cbb20fb6f8d2fdb4', 'tags_59e769a39a794784016db183', 'tags_5fb59f349a794750d2f066a4', 'tags_5579587b9a794765b93a40c1', 'tags_54f58ae49a79470e78b8c5a4', 'tags_59e769aa9a794784016dbd37', 'tags_54c19efb9a79476a788e32a3', 'tags_59e769a29a794784016db000', 'tags_5409f1bae063daa0f408aecc', 'tags_57a8b5b49a7947277fe446f4', 'tags_55928d0d9a794751dc82ffd3', 'tags_55928ee99a794751dc832ef4', 'tags_5592acc19a794719538d3107', 'tags_544f40e7cbb20f2fa7401b33', 'tags_5409f404e063daa0f408b50a', 'tags_5440a265cbb20f46adb2ed5f', 'tags_556e08df9a7947714806eae3', 'tags_559292b59a794751dc8385b7', 'tags_544f7102cbb20fda2807ffcb', 'tags_569245929a7947fc509e2593', 'tags_54b8dd8e9a79472d484d6ce0', 'tags_5500072e9a7947a5746c732a', 'tags_5592903d9a794751dc834fd3', 'tags_5620f7ad9a7947a9d4a0dafa', 'tags_551e81da9a794710d6b186e2', 'tags_591c342f9a7947272ca73e03', 'tags_55928eeb9a794751dc832f42', 'tags_543b979acbb20f86604809dc', 'tags_5409f3f5e063daa0f408b4d2', 'tags_596754d79a794772e48f6ac8', 'tags_578c189e9a7947793d10a491', 'tags_5409f42ae063daa0f408b5d7', 'tags_5453b4b6cbb20f837d98c6d2', 'tags_58384b4e9a7947b0d4162ed0', 'tags_5409f3a9e063daa0f408b372', 'tags_584f861b9a794796cc1ccf76', 'tags_585ca7089a794743fee6faba', 'tags_54a388cb9a7947516ea9e053', 'tags_54dca6bb9a794714bd831bb1', 'tags_542498c7cbb20feba0485f3b', 'tags_5447f8e2cbb20fd6cba57097', 'tags_545221d5cbb20f7096ffbe09', 'tags_5409f270e063daa0f408b0a6', 'tags_54b9ed399a79475078739e1a', 'tags_54255c4dcbb20f12fe13b04e', 'tags_55a4fefc9a79472c42d4e137', 'tags_543a7faecbb20f4fc9d2ad55', 'tags_5656a0209a79478074495e6f', 'tags_546a3320cbb20fe17cdc5661', 'tags_578ee9949a79473790f2f472', 'tags_547c0367cbb20f53267f5320', 'tags_5548b03b9a7947cf07a62c1d', 'tags_5425e055cbb20f23bc9f131d', 'tags_542b13eacbb20f552d46a1bb', 'tags_55f5879a9a7947cffb9658d1', 'tags_57b3f4ea9a79477b1cd7e3db', 'tags_5409f3c9e063daa0f408b44f', 'tags_543eef54cbb20f9f2cfd2044', 'tags_57ef04179a79473023d367f1', 'tags_5592aaa09a794719538d1886', 'tags_5469fbd8cbb20f2328bad11e', 'tags_548b0aba2ae59626f124a2c1', 'tags_5428324dcbb20f2cd90fafc1', 'tags_5409f20ae063daa0f408af88', 'tags_560434789a7947a16be29c94', 'tags_54747983cbb20f2248037288', 'tags_5432a369cbb20f3f1d83336d', 'tags_547d9fc5cbb20f7d39cad31b', 'tags_599af9279a79479c5b96b48c', 'tags_562df4cf9a7947167f62785e', 'tags_6193d2b39a7947797cb4e434', 'tags_549b60679a794728c74ac217', 'tags_54709a30cbb20fe23ccc7c31', 'tags_54a3c2e69a794765296d4b27', 'tags_559293329a794751dc838f0a', 'tags_5592a9189a794719538d05d1', 'tags_55805cde9a7947f0c9a6a603', 'tags_549179009a7947479780c4b1', 'tags_564bb3369a7947a19cee0f63', 'tags_5715f0099a7947c5a4b42d46', 'tags_55ec46da9a794711986c97f9', 'tags_546c8913cbb20f32bf9391e9', 'tags_553f88799a794702aa7f2c9a', 'tags_542d519acbb20ff48f967023', 'tags_56279f2a9a7947cacfc7160f', 'tags_60d97da69a79477a87c80bc8', 'tags_54fdbe249a79470f2b5e4af3', 'tags_545caa9ecbb20f366651641c', 'tags_549aa56f9a794714994f34aa', 'tags_5409f15ae063daa0f408ad90', 'tags_542a994dcbb20f1b9bbbca83', 'tags_624aaa569a7947d13afd02c0', 'tags_543ae0a8cbb20f36ab136d1f', 'tags_60001d189a79473cc91b256d', 'tags_5573f27d9a794712540f0d13', 'tags_5409f1d8e063daa0f408af0b', 'tags_542d4c4ecbb20faf2f73abda', 'tags_5409f3cee063daa0f408b462', 'tags_5446c96bcbb20f3824f54e73', 'tags_5409f420e063daa0f408b5a4', 'tags_567295799a794782b5801ea4', 'tags_5592b4ca9a7947de211fd6fe', 'tags_57a40bfe9a794750f3a5df2a', 'tags_547b114ccbb20f76d33728fc', 'tags_561cfd569a79472b6428fc11', 'tags_59c2b88a9a794747706c2942', 'tags_5453698ccbb20f4c0aebcdbf', 'tags_54eec5239a79470e4e7086aa', 'tags_5440237bcbb20f482df1ab5a', 'tags_599d4e349a79475c9d8d60dd', 'tags_5409f270e063daa0f408b0a8', 'tags_54dc84999a79477970b4fbb8', 'tags_559293969a794751dc8396bc', 'tags_54481ab1cbb20f7e25a1e612', 'tags_5444323ecbb20f0642a4120c', 'tags_5433772ccbb20ffb2cc157ab', 'tags_55f9967f9a79472d1398a7a5', 'tags_54cbae219a79470e8b1900bf', 'tags_597f135c9a79477b900aee7d', 'tags_59e769aa9a794784016dbd3b', 'tags_55928fe09a794751dc8346e2', 'tags_55155a7f9a79476438e357e2', 'tags_5409f292e063daa0f408b0fe', 'tags_544c1488cbb20fda6d160a87', 'tags_543c37e1cbb20f1d1f9e19e5', 'tags_542c02b8cbb20f5d06c1d876', 'tags_5409f145e063daa0f408ad2b', 'tags_54882bf5cbb20fa122acdf44', 'tags_6206a0e59a79478973cf165c', 'tags_5409f153e063daa0f408ad77', 'tags_54537a49cbb20faf3dd4834a', 'tags_59e769a49a794784016db47e', 'tags_544fbd5ccbb20f187460ac09', 'tags_54c0cf889a7947910758b80e', 'tags_5409f159e063daa0f408ad86', 'tags_6247ffd19a79472519f6323b', 'tags_544fe0accbb20f32b14924f0', 'tags_59e0736b9a79475ec91b4d63', 'tags_543446d0cbb20f158e77b4c7', 'tags_552cd8179a7947e9b5a8836f', 'tags_549ade979a79471dd35cd57d', 'tags_544e6fabcbb20f1b159b182e', 'tags_54ca2b2e9a79475a8f3beea1', 'tags_549d22d49a794774b9d4b461', 'tags_551a7ac19a7947a98cb1a6a4', 'tags_543253a9cbb20f0a674a8294', 'tags_54819afccbb20f3825ad5e90', 'tags_55abe3729a79473aa2828e9e', 'tags_542d5a68cbb20f1c43aa59e2', 'tags_54a6c52f9a79473840823979', 'tags_59e7699e9a794784016da9eb', 'tags_5409f41fe063daa0f408b5a1', 'tags_59e769ab9a794784016dc00b', 'tags_61a502c19a7947a816b1971b', 'tags_56493bb29a794785707d9004', 'tags_5472f5cfcbb20f1479cb0656', 'tags_5409f30ae063daa0f408b1e1', 'tags_587cd92c9a7947cfb0297a45', 'tags_5461cc05cbb20f4a9ff74c35', 'tags_5409f399e063daa0f408b322', 'tags_542e6babcbb20f0b4063b2ad', 'tags_5409f3bce063daa0f408b3eb', 'tags_543dc5a0cbb20f6ae0f6c93a', 'tags_54354834cbb20f14c07adfa8', 'tags_562cec809a79471d0b458f96', 'tags_5908c0929a79470ed787e77c', 'tags_59476a939a7947fb92550168', 'tags_58e539a09a79475a2b64a4ce', 'tags_56284db29a794744306b8663', 'tags_543e6d52cbb20fa8f2813058', 'tags_5425a5adcbb20f8dabc96b99', 'tags_54481740cbb20f6a6211709c', 'tags_55279d449a794780250fb50e', 'tags_56fe53c19a7947af38bd2b1c', 'tags_544c35e6cbb20fda6d160a8e', 'tags_548584bccbb20f0253603abe', 'tags_55fb09fd9a79470d5ca71e51', 'tags_5409f29ae063daa0f408b14b', 'tags_5559ea0a9a794766f4e636bc', 'tags_5656c8519a79471abd7d8aa4', 'tags_544a43c7cbb20fe507e4aa40', 'tags_54529489cbb20fb88f768ae2', 'tags_54b90ccc9a79473a6f82df1f', 'tags_547b0741cbb20f5b8c5ab185', 'tags_55928d4c9a794751dc83064e', 'tags_55f28aaf9a794740fdb86dae', 'tags_59190ef19a7947e878dc4154', 'tags_5462196bcbb20f54c18e06f9', 'tags_59e7699c9a794784016da552', 'tags_5409f2fce063daa0f408b1ce', 'tags_54dc848d9a79477970b4faf1', 'tags_564787b89a79478b9104da07', 'tags_545f117ecbb20f45c0ac905b', 'tags_54479a23cbb20f18bbac65b4', 'tags_57a47c6a9a794732adc4de89', 'tags_54dc85729a7947798a412232', 'tags_54b7ab379a794705610e5d9d', 'tags_569431139a7947f8fbdb497f', 'tags_62864fb99a7947ef26e01865', 'tags_595bbbd29a7947c5ce53a4c1', 'tags_5409f15de063daa0f408adac', 'tags_54dc857d9a7947798a4122d9', 'tags_542ab537cbb20f6dae167168', 'tags_610152559a79472a1a3b49ee', 'tags_5434adbacbb20f6882a37e5b', 'tags_58f5e5c69a79476ba8d2eb5a', 'tags_59e769ab9a794784016dbfc1', 'tags_54ded8c49a79471c341097f1', 'tags_547cd44ecbb20fb8dabdc8f3', 'tags_54dcaa6a9a79471bacf43a72', 'tags_543805b8cbb20f0da23860cc', 'tags_549422be9a79478f11d54521', 'tags_55ae408b9a79475c56cbf38a', 'tags_543c9ea5cbb20fd0c0129ee2', 'tags_5409f237e063daa0f408b000', 'tags_59e7699e9a794784016daa44', 'tags_574bb1a59a79474005285700', 'tags_5592914f9a794751dc8367cf', 'tags_55028bc59a794792df3a8fe7', 'tags_5691e9a69a7947cd7ee68f5a', 'tags_5409f406e063daa0f408b50f', 'tags_547322fdcbb20fb3387cfd50', 'tags_56f506fe9a79479749a82974', 'tags_54524b29cbb20f6bf16c6b48', 'tags_55928fab9a794751dc83416a', 'tags_542bb09fcbb20f14d61fabcb', 'tags_5409f39fe063daa0f408b33c', 'tags_628b7d279a7947d909a19cd5', 'tags_59e769a59a794784016db688', 'tags_5470a39acbb20fe23ccc7c33', 'tags_551d2e819a7947d0fd1a7819', 'tags_5409f14fe063daa0f408ad6b', 'tags_5409f3bae063daa0f408b3db', 'tags_55faec999a79474477751bae', 'tags_54a3c2e69a794765296d4b28', 'tags_5409f2dbe063daa0f408b188', 'tags_5a731ee69a7947556f9dc305', 'tags_544b3133cbb20fda9471d05f', 'tags_5581c42d9a794767c04ef630', 'tags_59ddfdcd9a794778d47547b7', 'tags_546502a4cbb20f3185719ac2', 'tags_59ea40349a79476a824ec18e', 'tags_543542ffcbb20f62476f298b', 'tags_624fdc8f9a79471b195c878f', 'tags_547d9fc5cbb20f7d39cad31a', 'tags_548d3b9e2ae596709f58ffb5', 'tags_628669339a7947ff0a6fa2f7', 'tags_54410483cbb20ffcdb9da22b', 'tags_55928fef9a794751dc834852', 'tags_55416a919a7947cfed7021b4', 'tags_60caed629a794749b1a8c594', 'tags_59e769a69a794784016db6af', 'tags_544c35e6cbb20fda6d160a8f', 'tags_5592a88c9a794719538d0213', 'tags_559fc9fe9a79476988e98095', 'tags_5592a7d99a794719538cf8a8', 'tags_55e04a349a7947b6eb9d4f7f', 'tags_5409f40fe063daa0f408b538', 'tags_602550909a7947de72059caa', 'tags_5628c3559a79470d9daa4dc2', 'tags_56b10a079a79474e4e6ae4b9', 'tags_5642f7439a7947031423bf13', 'tags_55928e519a794751dc832063', 'tags_542a63e3cbb20f55b72dd71f', 'tags_56534a129a794771fb9463a8', 'tags_5592a7919a794719538cf506', 'tags_5e1c5f109a794753505b7acb', 'tags_6218d8a99a7947b6c7a13fd2', 'tags_54ec51db9a794734d4e70259', 'tags_54dc854a9a7947798a41200e', 'tags_5447a529cbb20f19f7d993b6', 'tags_591fd16a9a79478313b41db8', 'tags_546ee48bcbb20f5eea50a524', 'tags_542a7d3fcbb20f470cdca569', 'tags_544f87c8cbb20f9821503f80', 'tags_5469f442cbb20f0e9246d0eb', 'tags_542c2650cbb20f8aba7354cd', 'tags_54494c35cbb20f6af303476d', 'tags_60425e749a79472f90d5244a', 'tags_549d80b59a794784813dc9eb', 'tags_54fb02a29a7947067891faf0', 'tags_54e3daa29a79479b5644070f', 'tags_5592b4499a7947de211fc963', 'tags_543f8ae3cbb20f22e8041e9e', 'tags_552e5ee39a79477b088017fa', 'tags_55368f289a7947b458e35fab', 'tags_5409f24ee063daa0f408b054', 'tags_58a0918f9a79473957d0c445', 'tags_551bd7f49a79470dbd52b5ef', 'tags_596a14479a79474b32bcf795', 'tags_5462352bcbb20f235c8efea9', 'tags_542fd025cbb20f30f49b0b03', 'tags_54cfb60b9a79478e97d69a83', 'tags_5592a7259a794719538cef20', 'tags_558548b79a7947f84b84475e', 'tags_54d4ee629a7947135c70f86a', 'tags_561622379a79472128ff8cda', 'tags_559297199a794751dc83cfd5', 'tags_54534719cbb20f2277f5b20c', 'tags_5428a3c1cbb20fd19019798f', 'tags_59b136c89a794769d14bfae2', 'tags_57ff98c09a794713de3af72b', 'tags_56cad4a29a79476ebd19d766', 'tags_55928da39a794751dc830f15', 'tags_5409f32be063daa0f408b21a', 'tags_54dde8399a79479157938272', 'tags_550076029a794704385a5854', 'tags_546072b6cbb20f71574e13dc', 'tags_542cbf8bcbb20f1efb10a974', 'tags_58f8eef19a79474169c029a8', 'tags_54243eb1cbb20f69cad24b21', 'tags_59e769a09a794784016dac0b', 'tags_54dca6a59a794714bd831a97', 'tags_5460fc46cbb20f7a80b8ede3', 'tags_54341782cbb20ff97344ab56', 'tags_54431a30cbb20f87865683ce', 'tags_577fbd0c9a79471b890bcbd9', 'tags_542a8a86cbb20fe4248d1a6f', 'tags_54af05599a79474704d74f3a', 'tags_626838d79a7947307fd3658e', 'tags_5444666fcbb20fafce492895', 'tags_55755e5c9a7947b830dc3bd1', 'tags_542a7fdccbb20f03dc151611', 'tags_55928c5b9a794751dc82efd3', 'tags_559289e39a79474deed83218', 'tags_5624fb499a794770eb25f4c9', 'tags_6244a4329a7947889aaaa573', 'tags_55928e739a794751dc8323fc', 'tags_5506f5bb9a79473776a89cee', 'tags_561e467f9a79475596034b24', 'tags_55a8996d9a7947eec69018e3', 'tags_55e724099a7947eded8b60ce', 'tags_54326b3acbb20fa300150c1c', 'tags_5409f289e063daa0f408b0d8', 'tags_57cfb2019a794773f936398e', 'tags_5656c0d59a794715242eb70a', 'tags_5980696d9a79472862235020', 'tags_592e595b9a79479e9811c755', 'tags_54789db0cbb20fa3248054a4', 'tags_5e8e61bc9a794761bd552bf9', 'tags_592567069a7947e1221d04e9', 'tags_549a85859a794710fb268819', 'tags_5424b92bcbb20f96c9a63b72', 'tags_55928f909a794751dc833f06', 'tags_59e769ac9a794784016dc078', 'tags_54b918699a79473b752126c7', 'tags_54b90cea9a79473ba31269dd', 'tags_5409f19ee063daa0f408ae88', 'tags_5592a6a09a794719538ce65c', 'tags_5e8e62239a794760aede58ac', 'tags_5409f1a1e063daa0f408ae92', 'tags_5507ede09a794736084bd0de', 'tags_5409f146e063daa0f408ad44', 'tags_592c0fe39a7947da98f8ec0a', 'tags_549450639a79479674545e08', 'tags_559294179a794751dc83a0d9', 'tags_5409f399e063daa0f408b319', 'tags_54c26c9e9a7947946a9adb4e', 'tags_560d29999a7947dbbdc462ef', 'tags_5450a4a1cbb20f53500b168b', 'tags_544ef24fcbb20f7c272187f1', 'tags_54bf77199a79475c6562b595', 'tags_5409f33ee063daa0f408b24c', 'tags_543fb924cbb20fc89b01bfea', 'tags_5481c8f3cbb20fb1c0713cc3', 'tags_547853a5cbb20f44c125af36', 'tags_5542467d9a794761f463ec89', 'tags_543e86ddcbb20f4ede3c3142', 'tags_59e769a29a794784016dafe5', 'tags_5549cdcd9a79476557dd0c13', 'tags_5a167d299a79477b9e7e2e72', 'tags_550129059a79475ad311197f', 'tags_542bf250cbb20f2739357eb0', 'tags_593bea6b9a794719f3978f9c', 'tags_548922402ae5965a8240f9fb', 'tags_55e7049d9a7947da71bb8b15', 'tags_59e7699e9a794784016daa32', 'tags_5592a8719a794719538d0091', 'tags_5430b863cbb20f5b50fb21d0', 'tags_599577cc9a794742dbf40dae', 'tags_563b09069a7947a9b1577569', 'tags_56b2e8e59a794772ded5fe29', 'tags_54c0a7359a794783e39dbb48', 'tags_5485e810cbb20fe31ee4165d', 'tags_54366e67cbb20f3e93c43db0', 'tags_563f31029a794752e2ec0e07', 'tags_54d08ef29a7947c247916dc8', 'tags_54473e0acbb20fcec25d4865', 'tags_574816159a7947d424d46df0', 'tags_559296099a794751dc83c120', 'tags_5592a5cf9a794719538cdb6c', 'tags_565dbf009a794700cbbffcca', 'tags_547702c0cbb20fd2497be3bf', 'tags_591abcc59a79474a13944600', 'tags_546ab636cbb20fbbde36bac0', 'tags_54dc41619a79475ac466fc89', 'tags_5428d8aecbb20f531f21a38d', 'tags_548eefd92ae59625dd065bab', 'tags_55a9091e9a7947852b907b1b', 'tags_5447f2dacbb20f9055c49efc', 'tags_5490734c2ae59643ea1bf7db', 'tags_542a7903cbb20f183451f3bc', 'tags_5409f17ce063daa0f408ae19', 'tags_5409f3c7e063daa0f408b439', 'tags_59e7699b9a794784016da49b', 'tags_5409f3ece063daa0f408b4ab', 'tags_549a51e19a79477c9a288f5f', 'tags_5430d4f2cbb20f0df1822e48', 'tags_542a9a09cbb20f2879a666c9', 'tags_5522f6bf9a79478637e235be', 'tags_54249c56cbb20feba0485f42', 'tags_569f73099a79471517cc2d51', 'tags_552d2a169a79473adb290f97', 'tags_562ab9b69a79474ab09a6c4c', 'tags_55184a579a794712d2502dfe', 'tags_5409f33de063daa0f408b245', 'tags_550bf3a59a79471632128b31', 'tags_5798cc2f9a79471b2e0ddc53', 'tags_54c75b289a79478d35dc91b5', 'tags_546f47b1cbb20f352168da2b', 'tags_55928c669a794751dc82f0e4', 'tags_59e769a09a794784016dac1c', 'tags_60001cfb9a79473cd0b147cd', 'tags_54dcaa6e9a79471bacf43ab2', 'tags_545abf63cbb20f390bffc4e1', 'tags_55928c679a794751dc82f10f', 'tags_55d2be249a79475dc73d52a5', 'tags_549e8d069a7947a403fec967', 'tags_547de82dcbb20f04036ceee7', 'tags_57d28f359a79475cf91b78f1', 'tags_544539facbb20f34e7c84dd8', 'tags_55d309939a79470be8c77d02', 'tags_54dc84909a79477970b4fb35', 'tags_577d481c9a79475098378db7', 'tags_59241bac9a79471fd82b6667', 'tags_5409f293e063daa0f408b109', 'tags_59e769a59a794784016db4d1', 'tags_555f33529a7947c4c86f99aa', 'tags_55d5dff59a7947e796c15db2', 'tags_54c22e739a79476c97ff0fe2', 'tags_5409f35ae063daa0f408b26f', 'tags_58e46a169a7947bc39960291', 'tags_5719f5b19a794716dd6319a5', 'tags_55928cd39a794751dc82fb89', 'tags_545c698dcbb20fef6aa0ce15', 'tags_54f831e59a79473636733476', 'tags_550981ae9a794749ec560b8e', 'tags_54999bd49a79477410f812c9', 'tags_54513d23cbb20f4812c149d6', 'tags_565059449a79475046a4f4eb', 'tags_5fb251bf9a794762b918bf7d', 'tags_54321d3ecbb20f07083b3ba8', 'tags_555cced29a7947e90b958b97', 'tags_5409f231e063daa0f408afec', 'tags_54dc21329a794755d40541f5', 'tags_593acd1b9a7947019e8ed20d', 'tags_560bf47c9a79479f26fa84be', 'tags_5592ac949a794719538d2f09', 'tags_57d09ad49a794762270ff017', 'tags_557045d99a79473fd3e29ebe', 'tags_551c229b9a79473fe5f3ab8b', 'tags_56f849e89a79473e6a7efe38', 'tags_54ad77159a79471ab35bab1b', 'tags_545e5dabcbb20f3647bb057f', 'tags_5409f1a9e063daa0f408aea7', 'tags_5409f222e063daa0f408afcf', 'tags_59e769a39a794784016db24f', 'tags_5409f140e063daa0f408ad1e', 'tags_550892559a7947bbe98ffac9', 'tags_554400149a794784a394846b', 'tags_544e489fcbb20fa840684f63', 'tags_559d21df9a7947b8b625a06d', 'tags_5427e1a6cbb20ff82f9c4e66', 'tags_59d75df29a79479daeb4f996', 'tags_570d2cf79a79472675cd7b2e', 'tags_56a3dd369a794746d2bf34f7', 'tags_54479fe3cbb20f5b4c9e0424', 'tags_5441ec06cbb20f265fe01386', 'tags_5592b44b9a7947e00d5b57ad', 'tags_546f6456cbb20ff4d0b66e98', 'tags_544fc342cbb20f41ba375453', 'tags_5409f3f7e063daa0f408b4de', 'tags_54d0bdbc9a7947d886a51391', 'tags_542e4375cbb20f11df695825', 'tags_5409f3e2e063daa0f408b486', 'tags_546e330ccbb20f044f7f2dd8', 'tags_5409f23be063daa0f408b010', 'tags_54f9a7909a79471008db21f8', 'tags_5409f223e063daa0f408afd1', 'tags_542bffc5cbb20f4274409dd4', 'tags_58246e5a9a7947489fbb38ba', 'tags_56dd876a9a7947594f56ef6a', 'tags_543663fccbb20f3f7f3e0aa3', 'tags_54a4155f9a794765296d4b47', 'tags_5a3526c89a7947b8636ff026', 'tags_589a730f9a794712689db6e5', 'tags_5999c7ac9a7947215015be86', 'tags_557980409a794788c4ea0738', 'tags_5459ac88cbb20fa88a8c87d2', 'tags_54481740cbb20f6a6211709a', 'tags_54b63a8c9a7947cd46bd3fbf', 'tags_5409f40ee063daa0f408b534', 'tags_5409f3c4e063daa0f408b425', 'tags_54aa6e9e9a7947b861cb1ec3', 'tags_5409f3dfe063daa0f408b47d', 'tags_5612a4c79a794717d45b631a', 'tags_59e769a29a794784016db043', 'tags_55029abd9a79479e1ed1f026', 'tags_59e769a89a794784016db994', 'tags_5574523b9a79473f1b56611d', 'tags_5534ed909a79470f10783df7', 'tags_59919ba49a7947aace8e8bcf', 'tags_5527d09a9a79473a03922bac', 'tags_621d65139a7947b70d7c767f', 'tags_55190bec9a79476dbbddcb0b', 'tags_54dc84889a79477970b4fa96', 'tags_548efd462ae59633283b8f91', 'tags_543e74a0cbb20f103dd48f25', 'tags_5592b43f9a7947de211fc86d', 'tags_55ffb3ca9a7947f057abc58d', 'tags_543c6688cbb20f7bea967953', 'tags_542bd53dcbb20f3af2b429c6', 'tags_583c0fc09a7947e4ac313bd4', 'tags_5409f147e063daa0f408ad4c', 'tags_5409f18be063daa0f408ae48', 'tags_559292919a794751dc8382d3', 'tags_564c90299a79477aa286c729', 'tags_5409f292e063daa0f408b0fb', 'tags_5474e97fcbb20f143f96b3c6', 'tags_5460f516cbb20f22ce2b7471', 'tags_54dca6df9a794714bd831d87', 'tags_55928f889a794751dc833e39', 'tags_59e769999a794784016da022', 'tags_5409f378e063daa0f408b2a7', 'tags_59e769ab9a794784016dbff6', 'tags_543671eccbb20f6995c1bfa8', 'tags_546ebaafcbb20f18a2520623', 'tags_5592a89f9a794719538d0306', 'tags_55f6a4719a79473f06fccbb2', 'tags_546370b3cbb20fd4a15af907', 'tags_58747b079a79473e25f9cc0f', 'tags_559292bf9a794751dc83866c', 'tags_54abaf259a7947cc053f526f', 'tags_5453b348cbb20f7524018a97', 'tags_559293969a794751dc8396b7', 'tags_5a0c4bb59a7947111afb441e', 'tags_5458ce8ccbb20fab59c7c323', 'tags_561c761a9a7947384fda04e0', 'tags_5447d670cbb20fe33c4a4142', 'tags_5409f20fe063daa0f408af9b', 'tags_551b61a09a79474abaf22e5c', 'tags_5431f06ecbb20fbb1cc0b6e6', 'tags_54762acacbb20f729eeb6701', 'tags_543cfc4ccbb20f706d21fd08', 'tags_54aea8709a794743ff181a06', 'tags_57c640579a794722dd859406', 'tags_546853eccbb20fcca21615c2', 'tags_561172b89a7947a2c494c6c0', 'tags_56e189fb9a79477bff9628eb', 'tags_5409f1c3e063daa0f408aee9', 'tags_5878cefb9a7947fcb16f2a94', 'tags_5432b641cbb20f290b43471c', 'tags_54bd1d219a794743b536303e', 'tags_55928eaa9a794751dc832922', 'tags_587b06969a79472888c17a72', 'tags_54f8a24a9a79478f5ce75a49', 'tags_5592a7409a794719538cf0da', 'tags_5409f3e8e063daa0f408b4a0', 'tags_61aa715f9a7947f4c8d39adc', 'tags_55928cbc9a794751dc82f91a', 'tags_559293a99a794751dc83984d', 'tags_545b0e48cbb20f08cb939e13', 'tags_54dc85729a7947798a412237', 'tags_545f7497cbb20fac41be05a6', 'tags_54ddf9569a794792cbc13f86', 'tags_546c9e6acbb20f655105098a', 'tags_542ac16fcbb20f6c334b06d8', 'tags_5409f1fce063daa0f408af5e', 'tags_54639ee2cbb20f16ace18c6a', 'tags_56e039369a79471665a0b63e', 'tags_553e475d9a79473860f89e90', 'tags_54c649989a79471f18b4190c', 'tags_54dea6cb9a7947c1e24ebe1f', 'tags_5650710d9a79475b10715ddb', 'tags_55c0cc7e9a7947cc764a4319', 'tags_5677c3709a7947bf4c8d550f', 'tags_5409f188e063daa0f408ae3c', 'tags_548d5ad72ae5966bd5424348', 'tags_542c2e29cbb20fe6248bcef1', 'tags_5448e98bcbb20f01a5844dbb', 'tags_54dc84ad9a79477970b4fcc1', 'tags_5409f262e063daa0f408b095', 'tags_55a600889a7947589618d934', 'tags_5555b9399a79476cf8036abf', 'tags_544ab66ecbb20f71ec42940d', 'tags_58273f289a7947364779dc09', 'tags_54db32969a79475909539643', 'tags_5409f176e063daa0f408ae06', 'tags_559293339a794751dc838f1d', 'tags_54cf13699a79473952fb3901', 'tags_5a0a2d2a9a794720e9994872', 'tags_54f88dfe9a79478548e8e93c', 'tags_54e136039a794756d3955160', 'tags_55254c949a794736eac16faf', 'tags_5444ccbacbb20f0e4acb1aa2', 'tags_552d44c29a79474b8bf48ae4', 'tags_542e4a24cbb20f4eb1d231a8', 'tags_544c1dfccbb20f5e81dd6800', 'tags_59e7699f9a794784016daac6', 'tags_5592b47c9a7947e00d5b5d18', 'tags_552e69b79a7947810951c0fd', 'tags_5a054c7c9a79478295abbbe5', 'tags_55e369af9a7947db501c535f', 'tags_58de3c0c9a7947e6260965a3', 'tags_5592abd89a794719538d2703', 'tags_5409f235e063daa0f408aff8', 'tags_54589d78cbb20f7111454a12', 'tags_55928dc39a794751dc831231', 'tags_55add68a9a7947fc6b111c35', 'tags_563c6a739a79475726d77b1b', 'tags_5473b4e8cbb20f2ac7414c51', 'tags_558d08699a794724284e206c', 'tags_5616951c9a7947b04ca600fb', 'tags_543452a9cbb20f9c392925e2', 'tags_5496f05e9a7947ecf9251c70', 'tags_5507efa89a7947379ba2938f', 'tags_5409f1b5e063daa0f408aebe', 'tags_5409f18be063daa0f408ae43', 'tags_54317392cbb20f451535f310', 'tags_54c390ec9a7947ea61ca85fe', 'tags_542ba6b3cbb20f973e76ec2c', 'tags_55928c5f9a794751dc82f04e', 'tags_5409f323e063daa0f408b20e', 'tags_54afcb049a7947629823e6ee', 'tags_54bd2da09a79470472562f4b', 'tags_54f6fc699a7947eed946b339', 'tags_5592ac2d9a794719538d2a68', 'tags_548a6c322ae59645c1233719', 'tags_542546d4cbb20fa4ab9bf9fe', 'tags_5592a88d9a794719538d0225', 'tags_55da229d9a7947443fe79cd1', 'tags_5447b9edcbb20f10a4d61a2a', 'tags_59bff6e79a7947110b633a08', 'tags_54477cf3cbb20f2b72ae89ca', 'tags_5863ab529a7947299b196075', 'tags_549be96c9a794744b2a9a9ce', 'tags_54d176ff9a79473ce0514716', 'tags_61c308f19a7947eff14a3c76', 'tags_54b312529a79474d9194e42c', 'tags_549ba69a9a794731cb7055b9', 'tags_54899c042ae596f6cc849243', 'tags_542d1ad5cbb20f64affe305a', 'tags_5a294baa9a7947f5204c49f3', 'tags_542660cecbb20f1c8d704327', 'tags_558436599a794770e10bde05', 'tags_56a7d7059a79475bb4b408dc', 'tags_5a001e949a79471414679ada', 'tags_54356f9acbb20f69df21dc37', 'tags_591553cf9a794704b9f5b788', 'tags_544c48c8cbb20f44ac472436', 'tags_620c8ff29a7947d04ff8e558', 'tags_595645029a79470d75fe8af5', 'tags_58abead89a7947fd2741ac19', 'tags_559295419a794751dc83b567', 'tags_549213e09a794754b85c7190', 'tags_5645206a9a7947845a703b04', 'tags_54efbb049a79470c7daed660', 'tags_5465fb19cbb20f03161396e7', 'tags_54e479949a79474bdf18b915', 'tags_57173e4a9a7947074ea21668', 'tags_557883709a79477861c90d04', 'tags_543f7ad2cbb20fa499a4c88c', 'tags_567901959a7947596ff6b32e', 'tags_59daf8769a7947e03dc50099', 'tags_5469cf98cbb20fd7ebe2d5c3', 'tags_5496b1369a7947df02d143cd', 'tags_56546aad9a7947ec515526fb', 'tags_5460c31ecbb20fc027d2e221', 'tags_5593ed979a7947add6b482c6', 'tags_58b6a79e9a7947eeff5adb0f', 'tags_5450c744cbb20f8bc5804d7d', 'tags_54394e22cbb20f0c496289dc', 'tags_55376ade9a7947255371a474', 'tags_59e769999a794784016da0c0', 'tags_5459051ecbb20f3c97e62f04', 'tags_554b65809a7947a9f98981e9', 'tags_542949e1cbb20fda92656699', 'tags_56d42f579a7947fb84247457', 'tags_550ad12b9a79477822c3b489', 'tags_593a987c9a79474cd2848874', 'tags_57039b679a79474490b76343', 'tags_5409f3abe063daa0f408b387', 'tags_60256f3d9a7947e87d674a94', 'tags_6030e8109a79475ba4390a26', 'tags_5409f41ce063daa0f408b582', 'tags_588a0fde9a7947d0eb22c891', 'tags_546c5d11cbb20fa1c8df533b', 'tags_6205059c9a79476bdb27197f', 'tags_5585109d9a7947e8db0d2bb0', 'tags_542c93d1cbb20f0c445fd0e2', 'tags_546b687bcbb20f06937090f3', 'tags_559296809a794751dc83c7b9', 'tags_55f91aee9a79474f42562bde', 'tags_5484cc18cbb20f92b796b142', 'tags_59e7699c9a794784016da5ea', 'tags_5448b9d1cbb20fbeeabcf750', 'tags_567813c99a79472690e75ea5', 'tags_542bd53dcbb20f3af2b429c3', 'tags_583c04229a7947d2df951536', 'tags_597390659a794723537982dc', 'tags_5409f299e063daa0f408b137', 'tags_5592b4459a7947de211fc91d', 'tags_5409f292e063daa0f408b0fd', 'tags_57df5bcf9a7947a0303d1f3e', 'tags_572721e19a794742a1bc5827', 'tags_54dc84b69a79477970b4fd42', 'tags_54cb1c569a7947b6e386e44d', 'tags_55928c659a794751dc82f0c8', 'tags_5506f76a9a79473d5d6ed294', 'tags_5409f1dde063daa0f408af1d', 'tags_5409f33ae063daa0f408b240', 'tags_55928fb19a794751dc8341de', 'tags_58ca77499a7947869b67ce6f', 'tags_5461f83acbb20f2c3c063f26', 'tags_56b8d2469a79479b7c58ab1b', 'tags_58da10d69a79476ebf63b9ae', 'tags_574377039a79478cfa6671f2', 'tags_5592b2eb9a7947d55bf51ea2', 'tags_5409f3e4e063daa0f408b48c', 'tags_5409f1e0e063daa0f408af34', 'tags_55bb8b8e9a79479a676611a1', 'tags_547c6ba4cbb20f7c96f57b46', 'tags_5644d14a9a7947559b7b39a3', 'tags_54374874cbb20f4aba0983f6', 'tags_620bad3d9a7947896d36e417', 'tags_557eba1d9a79474b9e1b61dd', 'tags_548ce59e2ae5961e413464cd', 'tags_54e472319a79473c89609edf', 'tags_5592a86f9a794719538d007e', 'tags_54e1f2e59a79477d77eeef3c', 'tags_561faf389a7947169d2ea21e', 'tags_5469d266cbb20fbe759b1055', 'tags_5640332d9a7947cdf316f2d4', 'tags_59e769989a794784016d9e81', 'tags_54dcaa5e9a79471bacf439cf', 'tags_5409f412e063daa0f408b54c', 'tags_548c0f152ae59680b1ca87d7', 'tags_542bbdcacbb20f1c64d1705e', 'tags_5465ca18cbb20f8addf2784f', 'tags_542a57c8cbb20f06f6b2fcaa', 'tags_5444b201cbb20fe71c64eb68', 'tags_5409f170e063daa0f408ade1', 'tags_5519493e9a79470313b8a719', 'tags_59e769a19a794784016dae2a', 'tags_560b300d9a794720f502227f', 'tags_577bdc739a79474bee4c8420', 'tags_5426b7accbb20fa6e4b12a97', 'tags_5592b4839a7947e00d5b5dea', 'tags_543fba8bcbb20fc89b01bfec', 'tags_555ca6d09a7947c508831e88', 'tags_59e769ac9a794784016dc080', 'tags_5592b4349a7947e00d5b5529', 'tags_5592a85d9a794719538cffb1', 'tags_542d039dcbb20f469a2bfbf5', 'tags_5409f168e063daa0f408adcf', 'tags_5492ffb89a79476f567053c4', 'tags_54d4615f9a7947be30d74b35', 'tags_54cfc56d9a7947994d6e8205', 'tags_542d6444cbb20f4c09274f1e', 'tags_556c5e759a794746ac71425a', 'tags_54dca6bb9a794714bd831bb8', 'tags_583d77cb9a794707b4f525df', 'tags_5409f140e063daa0f408ad1b', 'tags_559294ed9a794751dc83b00a', 'tags_56322c199a7947a9e656fb45', 'tags_620632469a7947538758b6bd', 'tags_589d720e9a794785a0950ad0', 'tags_59e769a39a794784016db0e3', 'tags_55928ec09a794751dc832b5a', 'tags_62478bef9a79470c3995da96', 'tags_55dc05f49a794736b3f143b0', 'tags_5592aa849a794719538d1745', 'tags_5592b43d9a7947de211fc846', 'tags_56210a619a7947be97aa721b', 'tags_54bf65259a79475a7cdc8062', 'tags_55928f349a794751dc83362b', 'tags_58e66fb09a794773c0a312aa', 'tags_59e1be919a79478492ff5249', 'tags_5592ac639a794719538d2ce0', 'tags_6051970c9a7947531e441dfd', 'tags_5492ef4b9a79476b3d9734a9', 'tags_54d342a49a79475663b8150d', 'tags_546227e6cbb20fd8d05686b3', 'tags_55be1f149a794766b9f9cefb', 'tags_542b2ff2cbb20f14719b1fd3', 'tags_597a0a6f9a79471ec6aa4872', 'tags_59e769a69a794784016db710', 'tags_54f79c4e9a7947654e1e7b26', 'tags_5409f244e063daa0f408b033', 'tags_5409f3fee063daa0f408b4f1', 'tags_560184d09a79475b26af1e4c', 'tags_54d4fd539a7947184f116800', 'tags_54f856469a794757d8fba9e1', 'tags_55b5fb959a794792d90074ec', 'tags_5409f378e063daa0f408b2a6', 'tags_551336439a7947250f1ffe9f', 'tags_5592a73f9a794719538cf0cb', 'tags_546de2b2cbb20f69a749355a', 'tags_59e769a49a794784016db48d', 'tags_54360b74cbb20f152f14e608', 'tags_54870fefcbb20f822c2348b8', 'tags_59ef8a709a79473e7d107422', 'tags_559291159a794751dc836293', 'tags_5551e1879a79477c170112c3', 'tags_54dd8aff9a79476e37beb15b', 'tags_543f58ebcbb20fef22c777d2', 'tags_545c04cfcbb20f30832b2df3', 'tags_553f3c609a7947d11373cc41', 'tags_59e769a19a794784016dad85', 'tags_5409f15ce063daa0f408ada2', 'tags_560e7c879a7947536149091b', 'tags_5425464acbb20f240250ba30', 'tags_544a56d9cbb20f33e1e3d66d', 'tags_5551d7619a794773484a3df1', 'tags_54cc1f1d9a794730e29a7e73', 'tags_56cd89669a7947219041a978', 'tags_592524719a7947ba0acee39f', 'tags_57eb801b9a79478a4014f154', 'tags_545e1d52cbb20f49da3e7cc0', 'tags_55f99fb99a7947bb1015bd4b', 'tags_5555ef809a7947092505ef9d', 'tags_56a0c7b29a7947e2fb4415e4', 'tags_543bc015cbb20fe185d5f929', 'tags_57ada2909a7947318b334594', 'tags_54caf8599a7947b081dfb817', 'tags_562e63d49a7947a7461b23ac', 'tags_55928ebf9a794751dc832b3b', 'tags_556c73ac9a794760d7eac580', 'tags_566c1c2b9a79473141f2904a', 'tags_577183969a79474f1c0bed32', 'tags_54f763f09a794759a6f44bda', 'tags_547dfca1cbb20f6b3ca1f564', 'tags_549453f39a7947961b09d23f', 'tags_55190bce9a7947698c1cf3b8', 'tags_5592a7069a794719538ced1d', 'tags_558411619a794757c41dc721', 'tags_54341782cbb20ff97344ab55', 'tags_54348d6ecbb20f744bb2049f', 'tags_56e2a7619a7947fab9232c06', 'tags_5664342e9a79470e799734d5', 'tags_54642726cbb20f100425faf0', 'tags_5409f3fae063daa0f408b4e2', 'tags_5926bba69a79472575932c56', 'tags_55e838889a794752b74278ae', 'tags_5935a0ec9a794711c795c324', 'tags_5692165a9a7947de69309215', 'tags_546a5e39cbb20fac62161242', 'tags_544252f7cbb20f5ec3044338', 'tags_5592b3419a7947d55bf52700', 'tags_57af269d9a7947da5eee2ead', 'tags_5592aa279a794719538d1315', 'tags_6171d4f79a7947f2ab558e0c', 'tags_575668199a7947c8f102c896', 'tags_546690d4cbb20f42013801f0', 'tags_5592948a9a794751dc83a909', 'tags_569f76ed9a794719d7a27919', 'tags_54d2787d9a79471e57710807', 'tags_54a51ed69a79476a1bc6176e', 'tags_5409f23be063daa0f408b013', 'tags_550189589a79470ea915f8fd', 'tags_549253869a794759d029fde7', 'tags_5568bd1e9a7947463170dd0f', 'tags_5489d6742ae5963c9380c2e6', 'tags_54ed8fa09a79475dc90f65fc', 'tags_543b9725cbb20f86604809d9', 'tags_565efae69a79479cefeb69b0', 'tags_55267b1a9a7947e749481c18', 'tags_55928e909a794751dc8326bf', 'tags_58a43d329a794739fedd7225', 'tags_5439409bcbb20f457c9fa915', 'tags_547aae21cbb20f78c7650afe', 'tags_5463187acbb20f627a790ea4', 'tags_54b0cd429a79470d4fbdf5a3', 'tags_578cd7389a794724e778ab37', 'tags_543da39fcbb20f2fe7e8a83a', 'tags_5e1c3f709a794740571b8872', 'tags_59e769a29a794784016db0ad', 'tags_54a32b2d9a7947489ff3a9cf', 'tags_5437cbeecbb20f277751dbb0', 'tags_54365e4bcbb20f044b83974d', 'tags_5465fdfccbb20f03161396f0', 'tags_54478a49cbb20f4d021a8dc1', 'tags_5951eff19a7947a846aad6ed', 'tags_55928d609a794751dc830850', 'tags_552e9cdf9a7947b219b1eabe', 'tags_58d454a69a7947867064cd6e', 'tags_57cf96be9a79470255928f84', 'tags_5409f3c2e063daa0f408b416', 'tags_59d58b419a794740c86a78c8', 'tags_55f2ab769a7947426fd2154f', 'tags_5409f18be063daa0f408ae45', 'tags_591da3369a7947713fe2fe1b', 'tags_5444cfb8cbb20f471a4d8822', 'tags_566ecf7c9a79470f6d9fa5a8', 'tags_54298881cbb20fe3a3d5bd80', 'tags_54c903289a7947746fbbac8c', 'tags_55071ad89a79475e8567829e', 'tags_5435597acbb20fe010d5063e', 'tags_558edec09a7947a65635f66b', 'tags_57bebef79a79476ade95f03e', 'tags_564afbef9a79474cac018c89', 'tags_553502849a79471ec3dd3478', 'tags_5592ab4e9a794719538d20d6', 'tags_543da98fcbb20fa86a2f74ad', 'tags_5447f4e0cbb20f9dfda9f90e', 'tags_588bc50e9a794720d8b1cefa', 'tags_58e7d0359a79478045a925a3', 'tags_55a7dc809a7947743532ffd4', 'tags_59e769a99a794784016dbc5b', 'tags_5424580ecbb20f7b9e104221', 'tags_6155488e9a7947529b960fdb', 'tags_542828cecbb20f466cf7c94d', 'tags_545fb0b7cbb20f23d0e0213e', 'tags_59e769a69a794784016db6da', 'tags_546de2b9cbb20f4906747932', 'tags_570cfb489a7947e33e50807d', 'tags_605197409a794753a58edc1f', 'tags_543d1b9bcbb20f52831af9ed', 'tags_555ec94b9a794772ac9714c4', 'tags_544e640bcbb20fa086430154', 'tags_563cbb5e9a7947b7cd21ed1c', 'tags_5592a9cd9a794719538d0ed2', 'tags_55d2fe9c9a79470494aef5c1', 'tags_542c161bcbb20f546c60be66', 'tags_54cf52539a7947537228d2fc', 'tags_5424974fcbb20ff68d5114c2', 'tags_54b938de9a7947403c321f19', 'tags_5614f1129a79479b4f7c0432', 'tags_54d31b549a7947357bd08568', 'tags_542c3a4dcbb20f752600abd0', 'tags_599fc6b79a79472bb9eeabd5', 'tags_543389cbcbb20f60c9252beb', 'tags_547cab2ccbb20f10e6f78407', 'tags_604dea639a79473af7381d6c', 'tags_548d656d2ae5969390dc236a', 'tags_54dc85829a7947798a412326', 'tags_54d3b1489a7947a052056ec0', 'tags_57a187939a79477749278ac5', 'tags_55928e869a794751dc8325c0', 'tags_54e9ae049a79472ce5b4f687', 'tags_59da14e69a794728450fabb0', 'tags_54c778079a79479f9d489ac6', 'tags_543ea068cbb20f0e99e72567', 'tags_54d9e0089a7947d2becd879f', 'tags_5693a2559a7947651ef8186e', 'tags_54dca6e09a794714bd831d95', 'tags_542491eacbb20feba0485f36', 'tags_565429fb9a79478e01e99046', 'tags_556841df9a7947765d6cf1a6', 'tags_545395f7cbb20f8c68051752', 'tags_547ddb22cbb20f66de23bec6', 'tags_570c99649a79475c1f614ec0', 'tags_54c855539a794718f4197b4a', 'tags_5434f40fcbb20fa2869468f5', 'tags_5795b6459a79471b5248e1e3', 'tags_542ba601cbb20f94b5f06047', 'tags_54882a66cbb20f31a4310cd8', 'tags_546c7977cbb20f489b11a3b0', 'tags_54266b96cbb20f9ecc13f083', 'tags_54d87b649a7947472d16c252', 'tags_5592adfe9a794719538d3ccf', 'tags_5409f15ce063daa0f408ad9e', 'tags_5953ecb89a79475e4f0c2b62', 'tags_56ce212e9a7947e03309ab1c', 'tags_5446d54dcbb20fbb25f92ea2', 'tags_546b4d52cbb20f84029d5163', 'tags_54d5d2569a79473001c64f9d', 'tags_5409f401e063daa0f408b4ff', 'tags_5592a83f9a794719538cfe13', 'tags_55c0b2bb9a7947aeb03f12a5', 'tags_58a5cb9c2ae596a9d2b67ee5', 'tags_5a1193759a7947851153c7e5', 'tags_555a6b8e9a7947e9305c54ab', 'tags_5460b080cbb20f86699e4032', 'tags_5948f7899a79478cb40e8a14', 'tags_56659bc89a794713ff81aafc', 'tags_55c6dd309a7947612978c643', 'tags_54341782cbb20ff97344ab54', 'tags_545c7347cbb20f4569aa8dfa', 'tags_57cd11cb9a794790eeaf58f5', 'tags_56c81a2d9a794755f3d464b9', 'tags_59d362479a79470891f41dc7', 'tags_54882438cbb20f0c07f3d28e', 'tags_578391f59a7947ed80dd3802', 'tags_54297537cbb20f767f4abc0b', 'tags_54325a34cbb20f1c452cdadb', 'tags_54dc85719a7947798a412224', 'tags_543df88dcbb20f4da0219860', 'tags_5497a96d9a7947fae31f89e0', 'tags_544fc342cbb20f41ba375454', 'tags_55a759049a794769f3d0dcaf', 'tags_543c16a4cbb20f52473a3f03', 'tags_5409f429e063daa0f408b5d1', 'tags_559290fb9a794751dc83606c', 'tags_547ec5ffcbb20fde0cc4dcac', 'tags_54edece49a79479d1042e162', 'tags_5409f29ae063daa0f408b152', 'tags_54743de6cbb20f84324d17f5', 'tags_61a0a60d9a7947f8b1ed3e47', 'tags_553759b79a794717880a639e', 'tags_59e769a59a794784016db532', 'tags_54b569ca9a7947b169a3b3c7', 'tags_5950bd189a7947742cb0489a', 'tags_5931ebc09a7947c5a33589d5', 'tags_5592ac879a794719538d2e74', 'tags_55993c469a7947c7c20e5923', 'tags_55928cc59a794751dc82fa0d', 'tags_5453a20ccbb20f8c68051757', 'tags_546c74cbcbb20f3b19bee035', 'tags_557d93169a7947b1d0fc8086', 'tags_5602b5549a794756f04260d4', 'tags_568bfa239a7947d3489d0e78', 'tags_5944cef69a794773709bb111', 'tags_544e5051cbb20f46de86aae0', 'tags_558d304c9a7947456b38fc14', 'tags_5592a91e9a794719538d0632', 'tags_543d5f3bcbb20f7fdfe3eeb0', 'tags_54350593cbb20f857b4befce', 'tags_556eb8289a79472f5c766030', 'tags_5592ad5f9a794719538d3747', 'tags_5409f39ee063daa0f408b337', 'tags_545f6b77cbb20f4422ca3ac6', 'tags_58e382679a79473773e11e85', 'tags_56f911ef9a794755e3f6d4e6', 'tags_56e172fe9a794763849c6aae', 'tags_55ae7b9f9a79473946c8026f', 'tags_5428d102cbb20f7b662eec28', 'tags_543f9523cbb20fbbe86fa897', 'tags_5537958f9a79474d05be3d25', 'tags_54269087cbb20fc76043a542', 'tags_573f046d9a794701d03ec30e', 'tags_5592a86b9a794719538d0050', 'tags_55ac02319a794753057f3883', 'tags_5409f3a2e063daa0f408b346', 'tags_59d1212b9a7947824a853cd0', 'tags_59e2e30d9a7947cc3a1b4b4f', 'tags_542d728ecbb20ff143b5d53a', 'tags_595951479a79470ca75fc67a', 'tags_56798f819a79472ca49765d8', 'tags_572214b29a794775784680e4', 'tags_5409f3afe063daa0f408b3a8', 'tags_5951eff19a7947a846aad6ec', 'tags_5ed519669a79476935581c5e', 'tags_59e4ec479a7947294055c32d', 'tags_54534ee5cbb20f1f55593865', 'tags_542b9cc3cbb20f1dbbccacb8', 'tags_6144cd0c9a7947268c252efd', 'tags_571b6d489a79475b83f3e1f4', 'tags_54eaebef9a7947763d3be692', 'tags_559290b69a794751dc835a37', 'tags_566ef9a79a794748ea85cb45', 'tags_59e769a99a794784016dbabf', 'tags_55d03b829a7947cbab827117', 'tags_59e769a19a794784016dae54', 'tags_5503602d9a79471de65f0b87', 'tags_549944829a79476315fe350d', 'tags_54dc85439a7947798a411fbe', 'tags_542e79b9cbb20fb0073f2ca5', 'tags_5448b9d1cbb20fbeeabcf751', 'tags_599f1a019a794775a7de7e48', 'tags_617b9cfe9a7947d03e56adb7', 'tags_5437017acbb20f0b1e03ce74', 'tags_5409f23de063daa0f408b020', 'tags_5409f330e063daa0f408b222', 'tags_554c6ca89a79472e564ee377', 'tags_54c9f95b9a79474af81f25b5', 'tags_55928e769a794751dc83243a', 'tags_54dc83a89a79477933cef2db', 'tags_542a8a92cbb20fe4248d1a71', 'tags_56c1aac09a7947652dabfde9', 'tags_543d9ae5cbb20f46cf658cc0', 'tags_59d47ab39a79479eaf9c0818', 'tags_56a096cf9a79479c62bfd5f9', 'tags_5426be0acbb20f0c544303a0', 'tags_5502d54d9a7947c25fea0ca0', 'tags_5443e809cbb20f424c0ddb12', 'tags_552e64bf9a79477a7d5fa018', 'tags_542697f2cbb20f1c487e1c20', 'tags_54dcaa619a79471bacf439f3', 'tags_57eca1d99a7947050f41e6d2', 'tags_6051972b9a79475346350cec', 'tags_5409f3bbe063daa0f408b3e0', 'tags_5592aa429a794719538d146c', 'tags_5527cdb99a79473a03922baa', 'tags_5409f32de063daa0f408b21e', 'tags_559291c99a794751dc83720d', 'tags_54dcc2b29a794784f20a7f09', 'tags_54c8cf2e9a7947460902bef8', 'tags_54eef56c9a79472dbc123b5a', 'tags_5501ce4b9a79474393f3fe60', 'tags_54dcaa6b9a79471bacf43a8c', 'tags_627b6c5d9a79477b8529237e', 'tags_550ad5ad9a7947780d3590ad', 'tags_591aa5089a79473b80f22585', 'tags_546e2292cbb20f670e0f60c3', 'tags_624705389a79476ad15e916d', 'tags_54b6867f9a7947dbb8d725ed', 'tags_5488df1d2ae596ce22a9b86d', 'tags_546c7130cbb20f0dbb4b7584', 'tags_55d7e9099a7947cd7696b2fe', 'tags_546e0376cbb20f4f288f98f2', 'tags_5470be8ecbb20fa2f4f67a5f', 'tags_542eb854cbb20f0d9d725555', 'tags_56212b269a7947f1d766d81d', 'tags_54686402cbb20f734a629c0a', 'tags_573828269a7947a7085471c4', 'tags_5409f1ece063daa0f408af45', 'tags_546437a9cbb20fdb47afb77f', 'tags_5409f14fe063daa0f408ad6c', 'tags_57e01c2f9a79474e2d511622', 'tags_5581ad369a794758bea7e143', 'tags_55928ca49a794751dc82f647', 'tags_546a158dcbb20ff816bcd2e5', 'tags_54da0a2a9a7947e16f529171', 'tags_55e7f29d9a7947032e27fad9', 'tags_5446dd97cbb20f3ad235a5dc', 'tags_5436ab66cbb20f05e8166497', 'tags_54c63f1b9a794715c4e23314', 'tags_58ad83139a794762138ca48b', 'tags_54621178cbb20f4d7fa82574', 'tags_5435dbcfcbb20ff507c9390d', 'tags_596787ec9a79479e4e16711a', 'tags_54441394cbb20f1c81202251', 'tags_543c8c8acbb20fd0c0129ede', 'tags_55bdc2949a79473c3b2ba176', 'tags_54264cfccbb20fb98e84336f', 'tags_559293d59a794751dc839bc8', 'tags_5719f5869a794719c9a26d87', 'tags_55140c089a79479dbdce8533', 'tags_576be7539a79470571da4b8e', 'tags_549691df9a7947e301785ebd', 'tags_55cd32d39a7947959d6bd808', 'tags_55a540639a7947788e77ef3d', 'tags_544666f4cbb20f8b7a7d3632', 'tags_5592a5a99a794719538cd8f3', 'tags_5463385fcbb20f0c73ca938c', 'tags_54dc858b9a7947798a412387', 'tags_59e7699d9a794784016da8c5', 'tags_545b68aecbb20f6e627e1555', 'tags_54483785cbb20f748331d2c9', 'tags_54eca6279a79476af92abd56', 'tags_561c55319a7947e679e839f5', 'tags_542894c3cbb20f42017deb29', 'tags_54d1e6429a79475322dee76c', 'tags_54c0cbfc9a79478e0c45ffa8', 'tags_54491922cbb20f9f201804d4', 'tags_548724f9cbb20f5bd27cc92e', 'tags_55928caa9a794751dc82f6fc', 'tags_54505e58cbb20f4df7033abd', 'tags_55ee65f79a79472c627a0ca2', 'tags_57a562de9a79470f00b82bf8', 'tags_543ba9abcbb20fc71c01fc8f', 'tags_5409f251e063daa0f408b05f', 'tags_5614bc639a79476e4c6515ce', 'tags_553f74739a7947ee3bc50fb5', 'tags_54e7247e9a7947955b5ce084', 'tags_54ccb08d9a794745cf77b776', 'tags_563355769a7947065b93150f', 'tags_54257c2fcbb20f1732a4050c', 'tags_5592aa2c9a794719538d134f', 'tags_5742bed79a7947f03d4492ba', 'tags_546318f5cbb20ff8d807b7bd', 'tags_545cc547cbb20facb3d97531', 'tags_5474b5c9cbb20f452297141b', 'tags_54f5ddfb9a794762ce54a173', 'tags_55928cc99a794751dc82fa53', 'tags_5926ce3f9a794733e4e68aa4', 'tags_55424f679a79476a3352abf2', 'tags_564f25f99a7947a91437a1d8', 'tags_5409f289e063daa0f408b0db', 'tags_54d100059a7947f9d8b5023c', 'tags_54ca3e839a79476d148ecee6', 'tags_54b51fc29a7947a4d027cac0', 'tags_581667709a79470a486d3469', 'tags_55f243b79a79476b429b4944', 'tags_54dcaa729a79471bacf43aed', 'tags_545b3421cbb20f220a4f4fab', 'tags_5496c0339a7947e86e9304dc', 'tags_56292f229a79479a98466708', 'tags_5592903c9a794751dc834fd2', 'tags_56f4b2949a794741458a272c', 'tags_545b231fcbb20f3e4102a0d3', 'tags_54bb75db9a79478224c760dd', 'tags_555adf219a79470d28331502', 'tags_54dca6c99a794714bd831c84', 'tags_556cad339a79478eb85e5d08', 'tags_54dfb86b9a794723a31d992f', 'tags_54ba9f3f9a7947688e2de475', 'tags_55928bb29a794751dc82e2f2', 'tags_542ce912cbb20f56834801ce', 'tags_563210b09a79477c295315ef', 'tags_54dc85749a7947798a41225c', 'tags_5587efa99a794791a41961e7', 'tags_5592907b9a794751dc8355a1', 'tags_5433b34ccbb20fc755c7f50e', 'tags_55fabb509a7947d7ff81715f', 'tags_548711fdcbb20fa5c740b01c', 'tags_5592b3e59a7947de211fbf8a', 'tags_552d26059a79473588413cdd', 'tags_54d82fd49a794734c8870a4b', 'tags_552bd1f79a794771cd1512f7', 'tags_554f4dd29a7947e5cb43535c', 'tags_5592b43e9a7947de211fc85e', 'tags_543f335ecbb20f851f9325ec', 'tags_54d341499a794757c27595bd', 'tags_578e17059a7947ee1b9490c1', 'tags_547a1837cbb20f3c46b6a9b3', 'tags_5592a6ef9a794719538ceb7b', 'tags_59e769a59a794784016db589', 'tags_5454a074cbb20f10210aed9d', 'tags_5428802ecbb20fc353bedd96', 'tags_5544af8e9a7947c409177315', 'tags_5551c4e29a7947662a8e42b3', 'tags_54dca6919a794714bd8319b9', 'tags_56d01a4c9a79471e7b47ffa1', 'tags_54342f18cbb20f7b0b69bb2f', 'tags_542c94fccbb20f3a7b29e9ce', 'tags_61ff7e379a79470e229441c9', 'tags_559290069a794751dc834a78', 'tags_555e527a9a794745e375b471', 'tags_56bc51889a79474bc38d2659', 'tags_5a0eee8a9a7947498ba0d835', 'tags_5718d92f9a7947a319b90cda', 'tags_559292709a794751dc837ff2', 'tags_555d4dc19a7947226cfe6cd6', 'tags_55928c919a794751dc82f459', 'tags_54b6f8229a7947e6f1c9dc34', 'tags_59e769a59a794784016db59b', 'tags_54249c56cbb20feba0485f41', 'tags_5409f15de063daa0f408ada6', 'tags_5460acb7cbb20f2c7b33da68', 'tags_54c0fb789a7947a8934428b2', 'tags_543fc7a1cbb20f866b8e6997', 'tags_5409f416e063daa0f408b56e', 'tags_559290bc9a794751dc835acc', 'tags_59e769a69a794784016db6b7', 'tags_59e769999a794784016da0f2', 'tags_54dca6a99a794714bd831ace', 'tags_54d09caa9a7947c80bc089ad', 'tags_55f95de39a79477392411389', 'tags_55928db39a794751dc8310ae', 'tags_54bd25859a794746c827c406', 'tags_5ee124549a7947e04304ce84', 'tags_54befda29a794748b7407f38', 'tags_5465ca18cbb20f8addf27850', 'tags_54465fdacbb20f2f790890b7', 'tags_54dc84959a79477970b4fb84', 'tags_57d040be9a7947649a2a72f8', 'tags_54b648ad9a7947d15de6c042', 'tags_54dca6d39a794714bd831ce4', 'tags_54b7e4cc9a79470f929d639c', 'tags_546b1aaacbb20f9e009f7398', 'tags_5440acb8cbb20f6bc91992ba', 'tags_555e526a9a79474da1adb0d2', 'tags_59e769a99a794784016dbc57', 'tags_54317ce6cbb20f21a10d8097', 'tags_55b4bc1e9a7947556dd836f4', 'tags_56160a5d9a794712fe96f910', 'tags_559296cd9a794751dc83cbe6', 'tags_54757f5dcbb20f391cbd2b23', 'tags_54dc85879a7947798a412347', 'tags_54278ea3cbb20f3064df104c', 'tags_5da86f909a79472888152aef', 'tags_561b9b809a79477d059b9baa', 'tags_54dca6c09a794714bd831c00', 'tags_55928e019a794751dc8318a3', 'tags_57980f1e9a794763e4807ad9', 'tags_542ab283cbb20f4f77ae484e', 'tags_54548b50cbb20f18e295e70f', 'tags_587f8a3c9a7947acc3800b80', 'tags_547f04b8cbb20f751c17a17f', 'tags_546cae49cbb20f6252032220', 'tags_5640c7139a79477fd671c78d', 'tags_5432632ccbb20f40e33a4802', 'tags_546f887ccbb20feec68a2d0b', 'tags_5c8e82659a7947825eae732f', 'tags_5a10cd039a7947373f85f9ff', 'tags_543b979acbb20f86604809db', 'tags_575676dd9a7947d6f0405ea5', 'tags_542d8200cbb20f2b8e4e28ab', 'tags_5460d566cbb20f6312fcdb86', 'tags_55928c5d9a794751dc82f02e', 'tags_5437d8b6cbb20f781997b8b2', 'tags_564b10bb9a79476726dc7b4b', 'tags_555dd3f29a794763fc99062a', 'tags_5592aafb9a794719538d1cf3', 'tags_5442ccfccbb20f57a125ee5b', 'tags_5445c977cbb20f7b064cdaba', 'tags_56d6eb219a7947cd0db8604a', 'tags_545fa4b4cbb20f4cbde07c89', 'tags_560af6ad9a794792af09a964', 'tags_5409f29be063daa0f408b156', 'tags_55efd3929a79470c60f94ad0', 'tags_5431c054cbb20f4513a783d5', 'tags_544b204fcbb20f5d6bcfeb9d', 'tags_5409f18ae063daa0f408ae3f', 'tags_54d207099a79475fd309d84c', 'tags_542e9ca3cbb20f44eb11fa2b', 'tags_5562e9389a7947ebae170076', 'tags_542e7c6acbb20f36d3a07ce6', 'tags_543eb58acbb20f0abe1799ed', 'tags_556173839a794745570a649f', 'tags_59e7699d9a794784016da763', 'tags_59e769a39a794784016db1fe', 'tags_5592b4639a7947e00d5b5a38', 'tags_54941bb49a79478ce4509c35', 'tags_5481ae73cbb20f8dfd3a9380', 'tags_542e7e88cbb20f52392e240d', 'tags_551a5ea99a79479481550007', 'tags_54a55be09a7947095c901e04', 'tags_54243eb1cbb20f69cad24b20', 'tags_57628ca79a7947483dcd8899', 'tags_59e769a39a794784016db1a1', 'tags_59e769a49a794784016db3e2', 'tags_54ce1e799a7947006954527d', 'tags_548efd462ae59633283b8f90', 'tags_55439a169a7947668ec67fe0', 'tags_59e7699d9a794784016da7f4', 'tags_542854c9cbb20f3a94cbbc72', 'tags_56b20b3c9a79472bf542e1bc', 'tags_552d40009a794749bf322f0c', 'tags_5409f1dae063daa0f408af13', 'tags_5474c3e2cbb20f593eedf1f5', 'tags_543bb660cbb20f74d0bb285e', 'tags_546dbcc2cbb20f0f6081419f', 'tags_5432f43ccbb20ff5b1ab5283', 'tags_545c6d69cbb20fa231f24388', 'tags_54696c6ccbb20f7e6aff7338', 'tags_544fb958cbb20f1ca05d9c4c', 'tags_5694c61e9a794758cf3639d4', 'tags_59db3a819a7947aa17d74b0f', 'tags_55197c029a794735bd6fd111', 'tags_55fbdc239a7947ffac5db80f', 'tags_54dc84a99a79477970b4fc86', 'tags_57fb8bb79a79472ca3c58366', 'tags_55928f7e9a794751dc833d57', 'tags_6068cefe9a7947d61ca4548d', 'tags_555c46449a79476d6130d09a', 'tags_5445aff2cbb20f18f0302de7', 'tags_59a903ec9a79477024dffcd1', 'tags_561f78eb9a7947c445e0700a', 'tags_54400e24cbb20f0d415e39b6', 'tags_5429527ecbb20f6382f94d74', 'tags_544624a6cbb20f4abbd95d37', 'tags_5578c3d89a79471f2560a3e4', 'tags_558b77ef9a794783faa033ef', 'tags_55928d349a794751dc8303f1', 'tags_5592aad79a794719538d1b1f', 'tags_5548fe319a79471342f2b41b', 'tags_563344b49a79477a13691ea3', 'tags_552e07109a79472f0ec51bd3', 'tags_5437b8cecbb20f71eb847521', 'tags_5499a81c9a794776553f0e4a', 'tags_543418c3cbb20ff97344ab5b', 'tags_5451ea9fcbb20f340d3062c0', 'tags_59e7699d9a794784016da7f5', 'tags_54b320b29a79475098d13628', 'tags_54771c6acbb20fa89ad0066c', 'tags_54450318cbb20fa7872b408f', 'tags_57430d5a9a794769ab0eaf43', 'tags_59f9d88b9a794725c8ce309c', 'tags_54739cc1cbb20fc57bc185e8', 'tags_5645ce1b9a7947faa12e9de6', 'tags_5449e32bcbb20faa72b31f97', 'tags_54dc84949a79477970b4fb69', 'tags_54d3456e9a794757c27595c0', 'tags_542cb3e6cbb20fdc9b064ee8', 'tags_5607e85e9a7947cd4c4249ef', 'tags_5592a6fb9a794719538cec5d', 'tags_5e8e60a39a7947606e1595b8', 'tags_5447ac1ccbb20f6e9bda6052', 'tags_551e87609a794713dfe8db13', 'tags_544e2f67cbb20f215a0d3290', 'tags_5409f1ece063daa0f408af47', 'tags_54873b39cbb20f74cdd13380', 'tags_549b91a79a79473069c2b757', 'tags_55116c4b9a7947931b76925f', 'tags_5a61f7899a79473bbb5f98ef', 'tags_560595179a794781daceb2f2', 'tags_543bf14fcbb20f2f5d160750', 'tags_5592b40b9a7947de211fc379', 'tags_5444af7ccbb20f53feabf6ab', 'tags_606c5a3a9a79474dadb7ec76', 'tags_550848d79a79478d321ec7af', 'tags_5498103c9a794716cbd1daa0', 'tags_55928d2f9a794751dc83037d', 'tags_542587b0cbb20fa55a5656d2', 'tags_5a1265529a79476250622254', 'tags_55f957709a794713212fcc28', 'tags_5409f38be063daa0f408b2c7', 'tags_58ff6eed9a794719505e7eee', 'tags_57b418f09a794707b76c5a32', 'tags_59e7699b9a794784016da3b7', 'tags_55928f3d9a794751dc8336ff', 'tags_5592b4959a7947de211fd116', 'tags_5409f396e063daa0f408b304', 'tags_6144b9089a7947207f2e429f', 'tags_544a4ee7cbb20f591694cb9e', 'tags_5431e5abcbb20f8943929fab', 'tags_564dde919a79472559315952', 'tags_549d25df9a794775979561d0', 'tags_5439577ccbb20f5e505dcecf', 'tags_54976b7b9a7947fa0e052f53', 'tags_54dc848e9a79477970b4fb0b', 'tags_54dc856d9a7947798a4121e9', 'tags_59e769999a794784016da099', 'tags_54c25d5e9a794787f243ce11', 'tags_5592a9389a794719538d0771', 'tags_5459d8cfcbb20f368a12bfe7', 'tags_542b3d10cbb20f47c07dcb56', 'tags_54f078db9a79479be43ab7ca', 'tags_59e769a99a794784016dbc72', 'tags_553f37289a7947c99194416e', 'tags_54255e8bcbb20f2a215db82c', 'tags_5450f281cbb20fb9bcf05f57', 'tags_557bee089a79470d81703873', 'tags_5491a1f89a79474adc26ce87', 'tags_575181de9a7947e0a652cfb7', 'tags_5880c5e79a794742f91f6357', 'tags_54e711939a79478dd43985d7', 'tags_542572c1cbb20f2a44a6accb', 'tags_543c08facbb20f4691adaaa0', 'tags_59e769ab9a794784016dc010', 'tags_556d70c89a7947e0875fece1', 'tags_56c7120f9a7947242ab41e65', 'tags_56ea69769a794745ba000c21', 'tags_54cf63709a79475d47a8ba3b', 'tags_54e484169a79474a8c0397d1', 'tags_54e3aaba9a7947971a2dca1a', 'tags_5821d79f9a794706defc7bee', 'tags_542c9a75cbb20f584f7eb1fa', 'tags_5409f29ae063daa0f408b141', 'tags_54351e4fcbb20fccd907d103', 'tags_5409f1c2e063daa0f408aee4', 'tags_54c867eb9a7947135fe5f99c', 'tags_54da3bf09a7947fd7ba3c792', 'tags_58ff1a0d9a79475435598cfa', 'tags_559291ff9a794751dc83768e', 'tags_59e769a49a794784016db34e', 'tags_54373a93cbb20fa9a5d4a2eb', 'tags_5860f91e9a79475a6edcd3f5', 'tags_55403b619a794785703f0451', 'tags_59879c9d9a79472cf59c13e9', 'tags_542a9417cbb20f472e50fe28', 'tags_54454b14cbb20f7668b12574', 'tags_542a7a40cbb20f28f0bf09ae', 'tags_5409f420e063daa0f408b5a5', 'tags_59e769999a794784016da08a', 'tags_5748bfbc9a79472ddf672c10', 'tags_572301b69a7947604d7b3545', 'tags_59e769a19a794784016dad4f', 'tags_5592a9159a794719538d05ac', 'tags_59b270b19a7947e8a4d5f36c', 'tags_55a6a3a79a794710fd514db6', 'tags_59e769a59a794784016db4d6', 'tags_54dcaa709a79471bacf43ada', 'tags_545d0a3fcbb20f2a0169613e', 'tags_54d9c5c29a7947c89c241f1d', 'tags_54dca68d9a794714bd831981', 'tags_5444c2cdcbb20fdb58bc80b0', 'tags_59bb61b59a7947e6906cea9d', 'tags_5592a9de9a794719538d0f8c', 'tags_549bce529a7947336d8312da', 'tags_55928d639a794751dc8308a7', 'tags_55189dc59a79472ff2b02fb4', 'tags_564236769a7947321b105c16', 'tags_55e40e2e9a794740f2f6f121', 'tags_5409f3dae063daa0f408b474', 'tags_5425b529cbb20f6f4fa55b6a', 'tags_542a2044cbb20f59f1b53063', 'tags_5a210bd39a794720286f78f3', 'tags_5409f1e4e063daa0f408af3a', 'tags_54ecae5a9a7947768b3bfe02', 'tags_54dc857e9a7947798a4122f0', 'tags_5426c112cbb20f7653ae7401', 'tags_551174169a7947a21b6eeb22', 'tags_549a80239a7947030d562c8a', 'tags_5478401ecbb20f79ca97c792', 'tags_562f327d9a794715007c8d03', 'tags_5978cdd19a79470868095459', 'tags_550699529a7947eff8a99d01', 'tags_5575626d9a7947bdbdb02fc5', 'tags_5409f40de063daa0f408b530', 'tags_54d22ee59a7947531b9169c2', 'tags_5409f382e063daa0f408b2bc', 'tags_557589289a7947dc4a0647cb', 'tags_59e7699a9a794784016da309', 'tags_54f460da9a7947db1775ac25', 'tags_553c02f69a79479e5c3a746e', 'tags_55f2918d9a79479c5ce46cb3', 'tags_5409f15ee063daa0f408adb1', 'tags_598d8b3d9a7947acd3d6416f', 'tags_559291239a794751dc8363d9', 'tags_55b442e99a7947268d204457', 'tags_57bc443f9a794727dfde7bfb', 'tags_552520d79a794711fa759c55', 'tags_54b6b3409a7947e1aa766cdc', 'tags_54d891d09a79474c60d36944', 'tags_59e769a49a794784016db4ae', 'tags_5459e845cbb20f0830c6abce', 'tags_59ef57dd9a7947099c279c44', 'tags_546df199cbb20f2f972ace2a', 'tags_5437fef0cbb20f94acc6cb50', 'tags_543671eccbb20f6995c1bfa7', 'tags_54d1ef649a7947532d68391b', 'tags_55573b749a7947a8ad194452', 'tags_571554329a79475a0f201c23', 'tags_599488359a7947eb5a8ab698', 'tags_57dbf3999a794754b56a2e03', 'tags_59e769a39a794784016db220', 'tags_567c359d9a79470765f5a7f4', 'tags_542c7fe7cbb20f269b710e2f', 'tags_553e16ec9a7947077dab6b42', 'tags_54dc84969a79477970b4fb8f', 'tags_59e769a59a794784016db555', 'tags_590840f09a7947af76059a2a', 'tags_5632084a9a794778ecb1ff06', 'tags_55ae69989a79472a20ad6be5', 'tags_559c0cfe9a7947028d3f07a1', 'tags_54450318cbb20fa7872b4091', 'tags_5909b37c9a794766b0131620', 'tags_55c3b2179a79473549c717f2', 'tags_5836e8809a7947d88aceb522', 'tags_54da2d219a7947f3a96aa0b0', 'tags_542546d4cbb20fa4ab9bf9fd', 'tags_54ecdc0a9a79470afc1192af', 'tags_559290e19a794751dc835e29', 'tags_5409f17fe063daa0f408ae25', 'tags_5812d4219a794731d7b71cae', 'tags_54b567c19a7947b192d17e51', 'tags_585c5ace9a79472f4fb6813b', 'tags_54dc84a49a79477970b4fc55', 'tags_56a88f449a7947f7552feff9', 'tags_546d8381cbb20f04ec4b9b8e', 'tags_620153719a7947220369cba5', 'tags_5436371acbb20fa784a26f10', 'tags_604dbd939a794731d06b141e', 'tags_54e4467c9a79471a1407460c', 'tags_55928ea49a794751dc83289b', 'tags_59ecc9109a794718e5eace02', 'tags_54340363cbb20f118b6c0b23', 'tags_543d9ae5cbb20f46cf658cc1', 'tags_574333299a79473b17d63f2e', 'tags_550c36d39a79473ea096d3da', 'tags_544bcba1cbb20fef45039e6e', 'tags_544ad219cbb20f765d90a5ff', 'tags_5480845bcbb20f85d50de877', 'tags_58490f1a9a7947eb89614a92', 'tags_59e769a09a794784016daaf0', 'tags_54a6595d9a79471e4bad7427', 'tags_56b0cae99a794707fa0537d1', 'tags_55ae1a569a79472f046b6d09', 'tags_59e769a09a794784016dab35', 'tags_543382c5cbb20f0ac213b6b0', 'tags_54255740cbb20f6b9b6d24c8', 'tags_5409f3c5e063daa0f408b42f', 'tags_5446094acbb20f752dd7af38', 'tags_58d8e98b9a7947c6a0ef9080', 'tags_5442df8dcbb20fcb3318f2d2', 'tags_569720269a79477547a80c6f', 'tags_55928bc19a794751dc82e487', 'tags_5409f2fce063daa0f408b1d3', 'tags_5409f3c0e063daa0f408b40d', 'tags_592ae7839a79475d187c8120', 'tags_543bd8a4cbb20f78726a6f6c', 'tags_5409f410e063daa0f408b53b', 'tags_542bdd97cbb20f0fa512c320', 'tags_555042889a79473334bd320b', 'tags_55928e729a794751dc8323cb', 'tags_59e769a49a794784016db368', 'tags_553681089a7947a7771b8c59', 'tags_559296de9a794751dc83ccaf', 'tags_54755852cbb20f1517d11f7b', 'tags_59e7699c9a794784016da58e', 'tags_5409f429e063daa0f408b5d0', 'tags_5fd3c03e9a794757eb32c513', 'tags_5592adc29a794719538d3b15', 'tags_54cea2f09a794715559f72c2', 'tags_56687d3f9a79474094d333dc', 'tags_54278a0ccbb20f1a0b3e493f', 'tags_560502c79a79473f0ca1d933', 'tags_54dca6bc9a794714bd831bc7', 'tags_552a147f9a7947ee8f3e388b', 'tags_542a8a92cbb20fe4248d1a72', 'tags_5786df6a9a79472a6ee87a24', 'tags_5592a8559a794719538cff4a', 'tags_55e9bdde9a7947410d0e682b', 'tags_5409f1b7e063daa0f408aec5', 'tags_5627fa949a7947274ec8e19e', 'tags_568a4cd19a79476eb3bd4e4e', 'tags_57359fd89a79473e908e0cc8', 'tags_542e28b3cbb20f586f7e2657', 'tags_55afd7ff9a7947464bc9c3ad', 'tags_5592ac8b9a794719538d2e9f', 'tags_5424d236cbb20f353dbe0376', 'tags_5592ac7c9a794719538d2dfe', 'tags_57334dac9a79476de216dcc7', 'tags_5436c770cbb20f4a1b2dfe62', 'tags_543d3beecbb20f6560383ae2', 'tags_5554a4449a7947cbfeca6c68', 'tags_54782bb8cbb20feb101d6d97', 'tags_594be3779a79472000cbfe5c', 'tags_54edb2e49a7947638f7688f3', 'tags_54808294cbb20fa3ce0d33c2', 'tags_54364dd7cbb20f35494f1f4d', 'tags_549168dd9a79474479d0dc20', 'tags_573ccac89a7947da459914e4', 'tags_59e769a99a794784016dbb17', 'tags_5487fb08cbb20f2026547043', 'tags_54610950cbb20f25fbaf36bd', 'tags_55928cb09a794751dc82f79f', 'tags_559290589a794751dc835257', 'tags_559297ea9a794751dc83da6d', 'tags_56fe2ef29a794780a5c4aeb4', 'tags_543e8eb4cbb20fad754a7011', 'tags_5808e3469a7947f70065ca98', 'tags_555b2dea9a7947442931f8b1', 'tags_5488ce5c2ae5967ffdac31db']

In [286]:
len(tags)

2570

In [287]:
v_x_cols=["authors","ctr","category","timestamp"]  +ents_feat_names+tags
v_y_cols=["views"] 

In [288]:
X = df_train[v_x_cols]
y = df_train[v_y_cols].values.ravel()

In [289]:
X.head()

,authors,ctr,category,timestamp,ents_LOC,ents_MISC,ents_ORG,ents_PER,tags_5409f196e063daa0f408ae65,tags_546e40e7cbb20f1d52dbaa55,...,tags_5487fb08cbb20f2026547043,tags_54610950cbb20f25fbaf36bd,tags_55928cb09a794751dc82f79f,tags_559290589a794751dc835257,tags_559297ea9a794751dc83da6d,tags_56fe2ef29a794780a5c4aeb4,tags_543e8eb4cbb20fad754a7011,tags_5808e3469a7947f70065ca98,tags_555b2dea9a7947442931f8b1,tags_5488ce5c2ae5967ffdac31db
document_id,,,,,,,,,,,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,560,1.5800000000,2,1649057384.0000000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,38,1.8530000000,0,1645167639.0000000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,560,0.0000000000,0,1644629042.0000000000,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,560,0.0000000000,0,1650623095.0000000000,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,560,0.0000000000,5,1650872543.0000000000,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [290]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [291]:
v_regr = RandomForestRegressor(random_state=0, max_depth=10, n_estimators=200)

In [292]:
v_regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=200, random_state=0)

In [293]:
pred = v_regr.predict(X_test)

In [294]:
score_views = r2_score(y_test, pred)
score_views

0.7695437234523311

In [295]:
importance_df = pd.DataFrame(v_regr.feature_importances_, columns=["value"], index=v_regr.feature_names_in_).sort_values(by="value", ascending=False)
importance_df

,value
ctr,0.2329521702
tags_5409f196e063daa0f408ae65,0.1982211624
tags_546e40e7cbb20f1d52dbaa55,0.1377395758
timestamp,0.1110510717
tags_5409f140e063daa0f408ad1d,0.0647493432
tags_5631bb949a794737108a5623,0.0450857647
authors,0.0229576098
tags_5e2af6549a794790a44be038,0.0183904344
tags_5409f15de063daa0f408ada7,0.0153491307
tags_5409f168e063daa0f408add3,0.0149232182


In [296]:
importance_df.to_csv(DATA_PATH/"imp.csv")

In [297]:
d_x_cols=["authors","ctr","category","timestamp", "text_length", "img", "video", "a", "p", "views", 'avg_sentence_len', 'max_sentence_len', 'min_sentence_len'] + ents_feat_names+tags
d_y_cols=["depth"]

In [298]:
X = df_train[d_x_cols]
y = df_train[d_y_cols].values.ravel()

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [300]:
d_regr = RandomForestRegressor(random_state=0, max_depth=70, n_estimators=500)

In [ ]:
d_regr.fit(X_train, y_train)

In [ ]:
d_pred = d_regr.predict(X_test)

In [ ]:
score_depth = r2_score(y_test, d_pred)
score_depth

In [ ]:
importance_df = pd.DataFrame(d_regr.feature_importances_, columns=["value"], index=d_regr.feature_names_in_).sort_values(by="value", ascending=False)
importance_df

In [ ]:
f_x_cols=["authors","ctr","category","timestamp", "text_length", "img", "video", "a", "p", "views", "depth", 'popularity'] + ents_feat_names+ tags
f_y_cols=["full_reads_percent"]

In [ ]:
X = df_train[f_x_cols]
y = df_train[f_y_cols].values.ravel()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
f_regr = RandomForestRegressor(random_state=0, max_depth=70, n_estimators=500)

In [ ]:
f_regr.fit(X_train, y_train)

In [ ]:
f_pred = f_regr.predict(X_test)

In [ ]:
score_frp = r2_score(y_test, f_pred)
score_frp

In [ ]:
score = 0.4 * score_views + 0.3 * score_depth + 0.3 * score_frp

score

In [ ]:
pd.options.display.float_format ='{:.10f}'.format
pd.set_option('display.max_rows',10000)

In [ ]:
importance_df = pd.DataFrame(f_regr.feature_importances_, columns=["value"], index=f_regr.feature_names_in_).sort_values(by="value", ascending=False)
importance_df

In [ ]:
X_test['pred_f'] = f_pred

In [ ]:
X_test['full_reads_percent'] =y_test

In [ ]:
X_test['delta'] = np.abs(X_test['pred_f'] -X_test['full_reads_percent'] )

In [ ]:
X_test.to_excel(DATA_PATH/"pred_f.xlsx")

In [ ]:
X_test.to_excel(DATA_PATH/"pred_f.csv")

In [ ]:
# from src.utils import write_to_file
# tags = [item for item in list(importance_df[importance_df["value"]>=0.0000001].index) if item.startswith("tags_")]
# write_to_file(DATA_PATH/"tags.txt", tags)

In [ ]:
# len(tags)

# Validation preparation

In [ ]:
cat_le = loads(DATA_PATH/"category.pickle")
df_test["category"]=cat_le.transform(df_test["category"])

In [ ]:
auth_le = loads(DATA_PATH/"authors.pickle")
auth_le_dict = dict(zip(auth_le.classes_, auth_le.transform(auth_le.classes_)))
df_test['authors'] = df_test['authors'].apply(lambda author: auth_le_dict.get(author, -1))

In [ ]:
df_test = df_test.apply(lambda row: str_to_list(row, 'tags'), axis=1)
tags_test=tags_le.transform(df_test["tags"])
tags_feat_names_test = ['tags_' + str(cls) for cls in list(tags_le.classes_)]
tags_df_test = pd.DataFrame(data=tags_test,
          index=df_test.index,
          columns=tags_feat_names_test)
df_test = df_test.merge(tags_df_test, left_index=True, right_index=True)
df_test = df_test.drop('tags', axis=1)

In [ ]:
df_test['day'] = pd.to_datetime(df_test['publish_date']).dt.strftime("%d").astype(int)
df_test['month'] = pd.to_datetime(df_test['publish_date']).dt.strftime("%m").astype(int)
df_test['hour'] = pd.to_datetime(df_test['publish_date']).dt.strftime("%H").astype(int)
df_test['hour_binary'] = df_test['hour'].apply(lambda row: row >= 7 & row <= 17).astype(int)

df_test['timestamp'] = df_test['publish_date'].apply(lambda d:  time.mktime(datetime.strptime(d, "%Y-%m-%d %H:%M:%S").timetuple()))

In [ ]:
df_test_ents = pd.read_csv(DATA_PATH /"df_text_test_ents.csv", index_col= 0)
df_test_ents = df_test_ents[['ents']]
df_test = df_test.merge(df_test_ents, left_index=True, right_index=True)
df_test = df_test.apply(lambda row: str_to_list(row, 'ents'), axis=1)

# ents_le = preprocessing.MultiLabelBinarizer()
ents_test=ents_le.transform(df_test["ents"])
ents_feat_names_test = ['ents_' + str(cls) for cls in list(ents_le.classes_)]
ents_df_test = pd.DataFrame(data=ents_test,
          index=df_test.index,
          columns=ents_feat_names_test)
df_test = df_test.merge(ents_df_test, left_index=True, right_index=True)
df_test = df_test.drop('ents', axis=1)

In [ ]:
df_test.columns

In [ ]:
# df_test['title_length'] = df_test['title'].str.len()

In [ ]:
v_X_validation = df_test[v_x_cols]

In [ ]:
v_X_validation.head()

In [ ]:
v_pred_validation = v_regr.predict(v_X_validation)

In [ ]:
v_validate_df = pd.DataFrame(data=v_pred_validation,
          index=v_X_validation.index,
          columns=v_y_cols)

In [ ]:
d_X_validation = df_test.merge(v_validate_df, left_index=True, right_index=True)
d_X_validation = d_X_validation[d_x_cols]
d_pred_validation = d_regr.predict(d_X_validation)

In [ ]:
d_validate_df = pd.DataFrame(data=d_pred_validation,
          index=d_X_validation.index,
          columns=d_y_cols)

In [ ]:
f_X_validation = df_test.merge(v_validate_df, left_index=True, right_index=True)
f_X_validation = f_X_validation.merge(d_validate_df, left_index=True, right_index=True)
f_X_validation = f_X_validation[f_x_cols]
f_pred_validation = f_regr.predict(f_X_validation)

In [ ]:
f_validate_df = pd.DataFrame(data=f_pred_validation,
          index=f_X_validation.index,
          columns=f_y_cols)
validate_df = v_validate_df.merge(d_validate_df, left_index=True, right_index=True)
validate_df = validate_df.merge(f_validate_df, left_index=True, right_index=True)
validate_df.to_csv(DATA_PATH/"validation.csv")
validate_df